In [ ]:
import numpy as np
import pandas as pd
import datetime
import boto3
from dateutil.relativedelta import relativedelta
from time import sleep
import logging
# !pip install lifelines
# from lifelines import KaplanMeierFitter
!pip install awswrangler
import awswrangler as wr
# from lib.data_loading_processing import *
# from lib.resupply_adherence_historical import *
# from lib.resupply_adherence_naive_forecast_separate import *

     |████████████████████████████████| 222 kB 8.1 MB/s            
     |████████████████████████████████| 43 kB 4.1 MB/s             
     |████████████████████████████████| 96 kB 9.9 MB/s             
     |████████████████████████████████| 207 kB 76.2 MB/s            
     |████████████████████████████████| 6.4 MB 52.1 MB/s            
  Attempting uninstall: lxml
    Found existing installation: lxml 4.6.4
    Uninstalling lxml-4.6.4:
      Successfully uninstalled lxml-4.6.4


In [ ]:
# Global parameters
s3_client = boto3.client('s3')
athena_client = boto3.client('athena')
bucket = 'project-ltv-data-analyst-bucket-prd'

# logging config
logging.basicConfig(filename='ltv_naive_forecast.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
##-------------------------------------------------------------------------
## 0 Run Athena query (utils)
##-------------------------------------------------------------------------
# sql query string
bt_resupply_final_202112 = """

with formatted_brightree_data as (
    select lower(airview_ecn) as airviewecn
        ,cast(so_key as double) as sokey
        ,cast(sodtl_key as double) as sodtlkey
        ,lower(order_type) as ordertype
        ,cast(so_create_date as date) as socreatedate
        ,cast(so_confirm_date as date) as soconfirmdate
        ,lower(purchase_rental) as purchase_rental
        ,upper(proc_code) as soproccode
        ,lower(manf_item_id) as manfitemid
        ,lower(item_id) as itemid
        ,lower(item_name) as itemname
        ,lower(category) as category
        ,cast(bill_qty as double) as qty
        ,lower(manufacturer) as manufacturer
        ,lower(supply_family) as supply_family
        ,(select max(cast(so_confirm_date as date)) from AwsDataCatalog.refreshed_brightree_for_ltv.brightree_data) as global_last_so_cf_dt
        ,cast(report_run_date as date) as report_generate_dt
    from AwsDataCatalog.refreshed_brightree_for_ltv.brightree_data
    -- where lower(airview_ecn) in ('19d908c5-0f42-4f93-b98c-269037edbcad','523e7757-3dd6-4e36-a8b6-5d28738cc3c2','0005e3b1-9031-4025-b2a9-e10539030128')
    where lower(airview_ecn) <> 'null'
    -- where airview_ecn = '523e7757-3dd6-4e36-a8b6-5d28738cc3c2'
    -- where airview_ecn = '0005e3b1-9031-4025-b2a9-e10539030128'
),

bt_pt_cnt_setup as (

    select airviewecn
        ,sum(case when ordertype = 'new setup' then 1 else 0 end) as cnt_setup
    from formatted_brightree_data
    group by airviewecn
),

sorted_brightree_data as(

    select *
        ,replace(replace(replace(supply_family_adj, ' for her', ''), ' mask', ''), ' cushion', '') as supply_family_new
        ,min(soconfirmdate) over (partition by airviewecn) as first_so_cf_dt
    from
    (select *
        ,case when supply_family in ('amaraview cushion') then 'amara view cushion'
              when supply_family in ('brevida pillow cushion') then 'brevida cushion'
              when supply_family in ('dreamwear ff cushion') then 'dreamwear full face cushion'
              when supply_family in ('dreamwear cushion') then 'dreamwear nasal cushion'
              when supply_family in ('n30i cushion') then 'airfit n30i cushion'
              when supply_family in ('dreamwear gel pillow cushions') then 'dreamwear gel pillow cushion'
              when supply_family in ('dreamwear pillows') then 'dreamwear gel pillow'
              when supply_family in ('wisp nasal mask fabric frame', 'wisp nasal mask clear frame') then 'wisp'
              else supply_family end as supply_family_adj
        ,case when category in ('ffm (cushion)', 'full face masks') then 'full face'
              when category in ('nasal (cushion)', 'nasal masks') then 'nasal'
              when category in ('pillow (cushion)', 'pillow masks') then 'pillow'
              else category end as category_adj
        ,dense_rank() over (partition by airviewecn order by soconfirmdate asc, sokey asc) as sokey_nm
    from formatted_brightree_data
    where qty between 1 and 6
    and soproccode in ('A4604', 'A7027', 'A7028', 'A7029', 'A7030', 'A7031', 'A7032', 'A7033', 'A7034', 'A7035', 'A7036', 'A7037', 'A7038', 'A7039', 'A7046',
                       'E0470', 'E0471', 'E0601', 'E0561', 'E0562')
    and airviewecn in (select airviewecn from bt_pt_cnt_setup where cnt_setup >= 1)
    )
),

bt_av_ptnt_setup_survival as (

    select /*+ BROADCAST(A) */ A.setup_date
                              ,B.*
    from
        (select lower(easy_care_number) as easy_care_number
            ,cast(setup_date as date) as setup_date
        from AwsDataCatalog.refreshed_brightree_for_ltv.patient) A
        join
        (select lower(ecn) as ecn
            ,is_dropout
            ,survival_days
            ,cast(censor_date as date) as censor_date
            ,compliance_status
            ,is_dropout_90
            ,survival_days_90
            ,cast(censor_date_90 as date) as censor_date_90
            ,avg_daily_use_90days
            ,avg_daily_use_q2
            ,avg_daily_use_q4
            ,days_wuse_90days
            ,days_wuse_q2
            ,days_wuse_q4
            ,days_wuse_4hrs_90days
            ,days_wuse_4hrs_q2
            ,days_wuse_4hrs_q4
            ,cast(last_record_date as date) as global_av_last_record_date
        from AwsDataCatalog.candpi_27000_curated_merged_tlzdidprd.av_ptnt_features_sc) B
        on A.easy_care_number = B.ecn
    where A.setup_date >= cast('2015-01-01' as DATE)
),

bt_data_av_ptnt as (

    select *
        ,case
            when days_confirm_first_so between 0 and 91 then 'q1'
            when days_confirm_first_so between 92 and 183 then 'q2'
            when days_confirm_first_so between 184 and 274 then 'q3'
            when days_confirm_first_so between 275 and 365 then 'q4'
            when days_confirm_first_so between 366 and 456 then 'q5'
            when days_confirm_first_so between 457 and 547 then 'q6'
            when days_confirm_first_so between 548 and 638 then 'q7'
            when days_confirm_first_so between 638 and 730 then 'q8'
            else 'after_2nd_year'
        end as quarter_confirm_first_so
        ,case
            when days_confirm_first_so between 0 and 365 then 'y1'
            when days_confirm_first_so between 366 and 730 then 'y2'
            when days_confirm_first_so between 731 and 1095 then 'y3'
            when days_confirm_first_so between 1096 and 1460 then 'y4'
            when days_confirm_first_so between 1461 and 1825 then 'y5'
            else 'after_5th_year'
        end as year_confirm_first_so
    from(
        select *
            ,date_diff('day', first_so_cf_dt, soconfirmdate) + 1 as days_confirm_first_so
        from sorted_brightree_data
        where airviewecn in (select ecn from bt_av_ptnt_setup_survival)
       )
),

resupply_ptnt_first_so as (

    select airviewecn as ecn_so
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('E0601') then qty else 0  end) as cpap_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7030') then qty else 0  end) as ff_mask_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7034') then qty else 0  end) as np_mask_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7027') then qty else 0  end) as on_mask_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7035') then qty else 0  end) as headgear_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7046') then qty else 0  end) as waterchamber_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7036') then qty else 0  end) as chinstrap_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_q1
         ,sum(case when quarter_confirm_first_so = 'q1' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_q1
         ,count(distinct(case when quarter_confirm_first_so = 'q1' then sokey else null end)) as cnt_so_q1

         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('E0601') then qty else 0  end) as cpap_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7030') then qty else 0  end) as ff_mask_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7034') then qty else 0  end) as np_mask_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7027') then qty else 0  end) as on_mask_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7035') then qty else 0  end) as headgear_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7046') then qty else 0  end) as waterchamber_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7036') then qty else 0  end) as chinstrap_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_q2
         ,sum(case when quarter_confirm_first_so = 'q2' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_q2
         ,count (distinct(case when quarter_confirm_first_so = 'q2' then sokey else null end)) as cnt_so_q2

         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('E0601') then qty else 0  end) as cpap_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7030') then qty else 0  end) as ff_mask_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7034') then qty else 0  end) as np_mask_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7027') then qty else 0  end) as on_mask_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7035') then qty else 0  end) as headgear_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7046') then qty else 0  end) as waterchamber_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7036') then qty else 0  end) as chinstrap_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_y1
         ,sum(case when year_confirm_first_so = 'y1' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_y1
         ,count(distinct(case when year_confirm_first_so = 'y1' then sokey else null end)) as cnt_so_y1

         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('E0601') then qty else 0  end) as cpap_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7030') then qty else 0  end) as ff_mask_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7034') then qty else 0  end) as np_mask_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7027') then qty else 0  end) as on_mask_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7035') then qty else 0  end) as headgear_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7046') then qty else 0  end) as waterchamber_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7036') then qty else 0  end) as chinstrap_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_y2
         ,sum(case when year_confirm_first_so = 'y2' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_y2
         ,count (distinct(case when year_confirm_first_so = 'y2' then sokey else null end)) as cnt_so_y2

         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('E0601') then qty else 0  end) as cpap_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7030') then qty else 0  end) as ff_mask_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7034') then qty else 0  end) as np_mask_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7027') then qty else 0  end) as on_mask_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7035') then qty else 0  end) as headgear_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7046') then qty else 0  end) as waterchamber_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7036') then qty else 0  end) as chinstrap_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_y3
         ,sum(case when year_confirm_first_so = 'y3' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_y3
         ,count (distinct(case when year_confirm_first_so = 'y3' then sokey else null end)) as cnt_so_y3

         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('E0601') then qty else 0  end) as cpap_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7030') then qty else 0  end) as ff_mask_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7034') then qty else 0  end) as np_mask_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7027') then qty else 0  end) as on_mask_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7035') then qty else 0  end) as headgear_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7046') then qty else 0  end) as waterchamber_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7036') then qty else 0  end) as chinstrap_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_y4
         ,sum(case when year_confirm_first_so = 'y4' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_y4
         ,count (distinct(case when year_confirm_first_so = 'y4' then sokey else null end)) as cnt_so_y4

         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('E0601') then qty else 0  end) as cpap_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7030') then qty else 0  end) as ff_mask_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7034') then qty else 0  end) as np_mask_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7027') then qty else 0  end) as on_mask_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7035') then qty else 0  end) as headgear_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7046') then qty else 0  end) as waterchamber_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7036') then qty else 0  end) as chinstrap_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_y5
         ,sum(case when year_confirm_first_so = 'y5' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_y5
         ,count (distinct(case when year_confirm_first_so = 'y5' then sokey else null end)) as cnt_so_y5

         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('E0601') then qty else 0  end) as cpap_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('E0470', 'E0471') then qty else 0  end) as bilevel_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7030') then qty else 0  end) as ff_mask_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7031') then qty else 0  end) as ff_cushion_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7034') then qty else 0  end) as np_mask_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7032','A7033') then qty else 0  end) as np_cushion_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7027') then qty else 0  end) as on_mask_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7028','A7029') then qty else 0  end) as on_cushion_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7035') then qty else 0  end) as headgear_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7037','A4604') then qty else 0  end) as tubing_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7046') then qty else 0  end) as waterchamber_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7036') then qty else 0  end) as chinstrap_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('E0561','E0562') then qty else 0  end) as humidifier_after_5th_year
         ,sum(case when year_confirm_first_so = 'after_5th_year' and soproccode in ('A7038','A7039') then qty else 0  end) as filter_after_5th_year
         ,count (distinct(case when year_confirm_first_so = 'after_5th_year' then sokey else null end)) as cnt_so_after_5th_year

         ,max(soconfirmdate) as ptnt_last_so_cf_dt
         ,max(global_last_so_cf_dt) as global_last_so_cf_dt
         ,max(report_generate_dt) as report_generate_dt

    from bt_data_av_ptnt
    group by airviewecn
),

bt_ptnt_features_first_so as (

    select airviewecn
        ,min(socreatedate) as first_so_cr_dt
        ,min(soconfirmdate) as first_so_cf_dt
        ,max(ordertype) as first_so_order_type
        ,max(case when soproccode in ('E0470', 'E0471', 'E0601') then 1 else 0 end) as flag_first_so_device
        ,max(case when soproccode in ('A7027', 'A7030', 'A7034') then 1 else 0 end) as flag_first_so_mask
        ,min(case when soproccode in ('E0470', 'E0471', 'E0601') then soproccode else null end) as first_so_device_hcpc
        ,min(case when soproccode in ('A7027', 'A7030', 'A7034') then soproccode else null end) as first_so_mask_hcpc
        ,min(case when soproccode in ('A7027', 'A7030', 'A7034') then category_adj else null end) as first_so_mask_category
        ,min(case when soproccode in ('A7027', 'A7030', 'A7034') then manufacturer else null end) as first_so_mask_manuf
        ,min(case when soproccode in ('A7027', 'A7030', 'A7034') then supply_family_new else null end) as first_so_mask_brand
    from bt_data_av_ptnt
    where sokey_nm = 1
    group by airviewecn
),

bt_ptnt_features_first_mask_cushion as (

    select airviewecn
        ,min(sokey_nm) as first_mask_cushion_sokey
        ,min(soconfirmdate) as first_mask_cushion_cf_dt
        ,min(soproccode) as first_mask_cushion_hcpc
        ,min(category_adj) as first_mask_cushion_cat
        ,min(supply_family_new) as first_mask_cushion_brand
    from(
        select *
            ,rank() over (partition by airviewecn order by sokey_nm asc) as sokey_nm2
        from bt_data_av_ptnt
        where soproccode in ('A7027', 'A7028', 'A7029', 'A7030', 'A7031', 'A7032', 'A7033', 'A7034')
        )
    where sokey_nm2 = 1
    group by airviewecn
),

bt_ptnt_features_mask_category_switch as (

    select *
        ,case when prev_mask_category is null or category_adj = prev_mask_category then 0 else 1 end as is_switch_category
    from(
        select *
            ,lag(category_adj) over(partition by airviewecn order by soconfirmdate) as prev_mask_category
        from bt_data_av_ptnt
        where soproccode in ('A7027', 'A7030', 'A7034')
        )
),

bt_ptnt_mask_category_switch_by_quarter AS (

    select airviewecn
         ,max(case when quarter_confirm_first_so = 'q1' then cnt_switch_category else null end) as cnt_switch_category_upto_q1
         ,max(case when quarter_confirm_first_so = 'q2' then cnt_switch_category else null end) as cnt_switch_category_upto_q2
         ,max(case when quarter_confirm_first_so = 'q3' then cnt_switch_category else null end) as cnt_switch_category_upto_q3
         ,max(case when quarter_confirm_first_so = 'q4' then cnt_switch_category else null end) as cnt_switch_category_upto_q4
         ,max(case when year_confirm_first_so in ('y1','y2') then cnt_switch_category else null end) as cnt_switch_category_upto_y2
         ,max(case when year_confirm_first_so in ('y1','y2','y3','y4','y5') then cnt_switch_category else null end) as cnt_switch_category_upto_y5
    from(
        select *
            ,sum(is_switch_category) over (partition by airviewecn order by soconfirmdate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cnt_switch_category
        from bt_ptnt_features_mask_category_switch
        )
    group by airviewecn
),

bt_ptnt_features_mask_brand_switch as (

    select *
        ,case when prev_mask_brand is null or supply_family_new = prev_mask_brand then 0 else 1 end as is_switch_brand
    from(
        select *
            ,lag(supply_family_new) over(partition by airviewecn order by soconfirmdate asc) as prev_mask_brand
        from bt_data_av_ptnt
        where soproccode in ('A7027', 'A7030', 'A7034')
        )
),

bt_ptnt_mask_brand_switch_by_quarter AS (

    select airviewecn
        ,max(case when quarter_confirm_first_so = 'q1' then cnt_switch_brand else null end) as cnt_switch_brand_upto_q1
        ,max(case when quarter_confirm_first_so = 'q2' then cnt_switch_brand else null end) as cnt_switch_brand_upto_q2
        ,max(case when quarter_confirm_first_so = 'q3' then cnt_switch_brand else null end) as cnt_switch_brand_upto_q3
        ,max(case when quarter_confirm_first_so = 'q4' then cnt_switch_brand else null end) as cnt_switch_brand_upto_q4
        ,max(case when year_confirm_first_so in ('y1','y2') then cnt_switch_brand else null end) as cnt_switch_brand_upto_y2
        ,max(case when year_confirm_first_so in ('y1','y2','y3','y4','y5') then cnt_switch_brand else null end) as cnt_switch_brand_upto_y5
    from(
        select *
            ,sum(is_switch_brand) over (partition by airviewecn order by soconfirmdate ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cnt_switch_brand
        from bt_ptnt_features_mask_brand_switch
        )
    group by airviewecn

),

bt_ptnt_so_features as (

    select A.*
        ,B.*
        ,D.first_mask_cushion_sokey
        ,D.first_mask_cushion_cf_dt
        ,D.first_mask_cushion_hcpc
        ,D.first_mask_cushion_cat
        ,D.first_mask_cushion_brand
        ,H.cnt_switch_category_upto_q1
        ,H.cnt_switch_category_upto_q2
        ,H.cnt_switch_category_upto_q3
        ,H.cnt_switch_category_upto_q4
        ,H.cnt_switch_category_upto_y2
        ,H.cnt_switch_category_upto_y5
        ,I.cnt_switch_brand_upto_q1
        ,I.cnt_switch_brand_upto_q2
        ,I.cnt_switch_brand_upto_q3
        ,I.cnt_switch_brand_upto_q4
        ,I.cnt_switch_brand_upto_y2
        ,I.cnt_switch_brand_upto_y5
        ,J.*

    from bt_ptnt_features_first_so A
    left join resupply_ptnt_first_so B
    on A.airviewecn = B.ecn_so
    left join bt_ptnt_features_first_mask_cushion D
    on A.airviewecn = D.airviewecn
    left join bt_ptnt_mask_category_switch_by_quarter H
    on A.airviewecn = H.airviewecn
    left join bt_ptnt_mask_brand_switch_by_quarter I
    on A.airviewecn = I.airviewecn
    left join bt_av_ptnt_setup_survival J
    on A.airviewecn = J.ecn
 )

select * from
    (
    select *
        ,date_diff('day', setup_date, first_so_cf_dt) as days_diff_setup_first_so_cf
    from bt_ptnt_so_features
    where first_so_order_type = 'new setup' and flag_first_so_device = 1 and flag_first_so_mask = 1
    )
where days_diff_setup_first_so_cf between -30 and 30;
"""

In [ ]:
##-------------------------------------------------------------------------
## 1 Data load and process (utils)
##-------------------------------------------------------------------------
def load_csv_from_s3(bucket, folder_src):

    # locate the csv file (user level summary of brightree and airview) generated by Athena
    s3_client = boto3.client('s3')
    objects_src = s3_client.list_objects_v2(Bucket=bucket, Prefix=folder_src)
    files_list = objects_src.get('Contents')
    csv_keys = [f['Key'] for f in files_list if f['Key'].endswith('.csv')]

    # load the csv file
    try:
        if (len(csv_keys) == 0):
            raise ValueError('No csv file.')
        elif (len(csv_keys) > 1):
            raise ValueError('Multiple csv files.')
        else:
            data_location = 's3://{}/{}'.format(bucket, csv_keys[0])
            df_hist = pd.read_csv(data_location)
    except ValueError as e:
        print(e)
    else:
        return df_hist

def process_resupply_data(df):

    # check difference between setup date and first so confirm date
    df['first_so_cf_dt'] = pd.to_datetime(df['first_so_cf_dt']).dt.date
    df['setup_date'] = pd.to_datetime(df['setup_date']).dt.date
    # df['diff_setup_1st_so'] = (df['first_so_cf_dt'] - df['setup_date']).dt.days
    # cohort selection
    df_selected = df[(df['first_so_order_type'] == 'new setup') & (df['flag_first_so_device'] >= 1)
                        & (df['flag_first_so_mask'] >= 1) & (df['first_so_cf_dt'] >= datetime.date(2015, 1, 1))]
    df_selected = df_selected[(df_selected['days_diff_setup_first_so_cf'] >= 0) & (df_selected['days_diff_setup_first_so_cf'] <= 30)]
    # add setup quarter and month
    df_selected['quarter'] = pd.PeriodIndex(df_selected.setup_date, freq='Q')
    df_selected['setup_month'] = pd.PeriodIndex(df_selected.setup_date, freq='M')

    # define mask brand switch flag
    df_selected['flag_same_mask_brand_first_five_years'] = np.where(df_selected['cnt_switch_brand_upto_y5'] == 0, 1, 0)
    df_selected['flag_same_mask_brand_first_two_years'] = np.where(df_selected['cnt_switch_brand_upto_y2'] == 0, 1, 0)
    df_selected['flag_same_mask_category_first_five_years'] = np.where(df_selected['cnt_switch_category_upto_y5'] == 0, 1, 0)
    df_selected['flag_same_mask_category_first_two_years'] = np.where(df_selected['cnt_switch_category_upto_y2'] == 0, 1, 0)

    return df_selected

def find_new_mask_brand(df_selected, mask_infor, count_size=100):
    # new mask brand is determined by the condition below
    # setup count in last three months > 100 and setup count in three months' time earlier < 100

    # get unique mask brand names from major manufacturers
    mask_brand_all = df_selected[['first_so_mask_brand', 'first_so_mask_manuf', 'first_so_mask_category']].drop_duplicates()
    mask_brand_major = mask_brand_all.loc[mask_brand_all['first_so_mask_manuf'].isin(['resmed', 'fisher & paykel', 'philips respironics'])]
    mask_brand_major = mask_brand_major.loc[mask_brand_major['first_so_mask_category'].isin(['full face', 'pillow', 'nasal'])]

    # three month before last complete month
    last_so_dt = min(pd.to_datetime(df_selected['global_last_so_cf_dt'])).replace(day=1)
    dt_three_month_earlier = last_so_dt + relativedelta(months=-3)
    # six month before last complete month
    dt_six_month_earlier = last_so_dt + relativedelta(months=-6)
    var_groupby = ['first_so_mask_brand','first_so_mask_manuf','first_so_mask_category']

    ind = (df_selected['setup_date'] >= dt_six_month_earlier) & (df_selected['setup_date'] < dt_three_month_earlier) & (df_selected['first_so_mask_brand'].isin(mask_brand_major['first_so_mask_brand']))
    cnt_mask_brand_before = df_selected.loc[ind,var_groupby].groupby(var_groupby).size().to_frame('cnt_before')
    ind = (df_selected['setup_date'] >= dt_three_month_earlier) & (df_selected['first_so_mask_brand'].isin(mask_brand_major['first_so_mask_brand']))
    cnt_mask_brand_after = df_selected.loc[ind,var_groupby].groupby(var_groupby).size().to_frame('cnt_after')

    cnt_mask_brand = cnt_mask_brand_before.join(cnt_mask_brand_after, how ='outer').reset_index()
    cnt_mask_brand['cnt_before'] = cnt_mask_brand['cnt_before'].fillna(0).astype('Int64')
    cnt_mask_brand['cnt_after'] = cnt_mask_brand['cnt_after'].fillna(0).astype('Int64')
    cnt_mask_brand['cnt_total'] = cnt_mask_brand.loc[:,['cnt_before', 'cnt_after']].sum(axis=1).astype('Int64')

    cnt_mask_brand_update = []
    for mask in list(cnt_mask_brand['first_so_mask_brand'].unique()):
        df_tp = cnt_mask_brand.loc[cnt_mask_brand['first_so_mask_brand'] == mask].sort_values('cnt_total', ascending = False).head(1)
        cnt_mask_brand_update.append(df_tp)
    cnt_mask_brand_update = pd.concat(cnt_mask_brand_update)
    new_mask_brand_infor = cnt_mask_brand_update.loc[(cnt_mask_brand_update['cnt_before'] < count_size) & (cnt_mask_brand_update['cnt_after'] >= count_size)]

    # remove those already in mask_infor
    new_mask_brand_infor = new_mask_brand_infor.loc[~new_mask_brand_infor['first_so_mask_brand'].isin(mask_infor['MaskBrandLower'])]

    return new_mask_brand_infor

def add_new_mask_brand(mask_infor, new_mask_brand_infor):

    new_mask_brand_infor['MaskBrand'] = new_mask_brand_infor['first_so_mask_brand'].str.title()
    new_mask_list = []
    for i in range(new_mask_brand_infor.shape[0]):
        new_mask_tp = {}
        brand_name_tp = new_mask_brand_infor.iloc[i].MaskBrand
        # eg 'Airtouch N20' to 'AirTouch N20'
        new_mask_tp['MaskBrand'] = np.where(brand_name_tp.startswith('Air'),
                                            brand_name_tp[:3] + brand_name_tp[3].upper() + brand_name_tp[4:], brand_name_tp)
        new_mask_tp['MaskBrandLower'] = new_mask_brand_infor.iloc[i].first_so_mask_brand
        manf_tp = new_mask_brand_infor.iloc[i].first_so_mask_manuf
        new_mask_tp['MaskManuf'] = np.where(manf_tp == 'resmed', 'ResMed',
                                            np.where(manf_tp == 'fisher & paykel', 'Fisher&Paykel', 'Philips'))
        cat_tp = new_mask_brand_infor.iloc[i].first_so_mask_category
        new_mask_tp['MaskCategory'] = np.where(cat_tp == 'full face', 'FF Mask',
                                            np.where(cat_tp == 'nasal', 'Nasal', 'Pillow'))

        new_mask_list.append(new_mask_tp)
    df_new_mask_brand_infor = pd.DataFrame(new_mask_list)
    mask_infor = mask_infor.append(df_new_mask_brand_infor)

    return mask_infor

def add_mask_launch_month(resupply_setup_month, mask_infor, count_size=10):

    # get approximate new mask brand launch date (first month with patient count > 10)

    for mask_brand in list(new_mask_brand_infor['first_so_mask_brand']):
        ind = (resupply_setup_month['first_so_mask_brand'] == mask_brand) & (resupply_setup_month['count_y1'] > count_size)
        if (sum(ind) > 0):
            df_mask = resupply_setup_month.loc[ind, ['setup_month','first_so_mask_brand']].sort_values(['setup_month'], ascending=False).reset_index(drop=True)
            df_mask_launch_tp = df_mask.head(1)
            mask_infor[mask_infor.loc['MaskBrandLower'] == mask_brand, 'LaunchMonth'] = df_mask_launch_tp['setup_month']

    return mask_infor

In [ ]:
##-------------------------------------------------------------------------
## 2 Generate historical resupply and adherence (utils)
##-------------------------------------------------------------------------
def count_resupply_yearly(data, freq_var='quarter', group_var='first_so_mask_brand'):
    # this function compute yearly resupply count group by freq_var

   # first year
    resupply_var_y1 = ['airviewecn', freq_var, group_var, 'cpap_y1', 'bilevel_y1', 'ff_mask_y1',
                        'ff_cushion_y1', 'np_mask_y1', 'np_cushion_y1', 'on_mask_y1',
                        'on_cushion_y1', 'headgear_y1', 'tubing_y1', 'waterchamber_y1',
                        'chinstrap_y1', 'humidifier_y1', 'filter_y1']
    df_y1 = data[resupply_var_y1]
    summary1 = df_y1.groupby([freq_var, group_var]).sum().astype('Int64')
    summary_compliance = data[['airviewecn', freq_var, group_var, 'compliance_status']].groupby([freq_var, group_var])\
                         .agg({'airviewecn':'size', 'compliance_status':'mean'})
    summary1= pd.merge(summary_compliance, summary1, how='left', on=[freq_var, group_var])
    summary1 = summary1.rename(columns={'airviewecn':'count_y1', 'compliance_status':'compliance'})

    summary1['pap_y1'] = summary1.loc[:, ['cpap_y1', 'bilevel_y1']].sum(axis=1)
    summary1['mask_y1'] = summary1.loc[:, ['ff_mask_y1', 'np_mask_y1', 'on_mask_y1']].sum(axis=1)
    summary1['cushion_y1'] = summary1.loc[:, ['ff_cushion_y1', 'np_cushion_y1', 'on_cushion_y1']].sum(axis=1)

    # second year
    resupply_var_y2 = ['airviewecn', freq_var, group_var, 'cpap_y2', 'bilevel_y2', 'ff_mask_y2',
                             'ff_cushion_y2', 'np_mask_y2', 'np_cushion_y2', 'on_mask_y2',
                             'on_cushion_y2', 'headgear_y2', 'tubing_y2', 'waterchamber_y2',
                             'chinstrap_y2', 'humidifier_y2', 'filter_y2']
    df_y2 = data.loc[data['cnt_so_y2'] > 0, resupply_var_y2]
    summary2 = df_y2.groupby([freq_var, group_var]).sum().astype('Int64')
    summary2_count = df_y2[['airviewecn', freq_var, group_var]].groupby([freq_var, group_var]).nunique()
    summary2= pd.merge(summary2_count, summary2, how = 'left', on = [freq_var, group_var])
    summary2 = summary2.rename(columns={'airviewecn':'count_y2'})

    summary2['pap_y2'] = summary2.loc[:, ['cpap_y2', 'bilevel_y2']].sum(axis=1)
    summary2['mask_y2'] = summary2.loc[:, ['ff_mask_y2', 'np_mask_y2', 'on_mask_y2']].sum(axis=1)
    summary2['cushion_y2'] = summary2.loc[:, ['ff_cushion_y2', 'np_cushion_y2', 'on_cushion_y2']].sum(axis=1)

    # third year
    resupply_var_y3 = ['airviewecn', freq_var, group_var, 'cpap_y3', 'bilevel_y3', 'ff_mask_y3',
                             'ff_cushion_y3', 'np_mask_y3', 'np_cushion_y3', 'on_mask_y3',
                             'on_cushion_y3', 'headgear_y3', 'tubing_y3', 'waterchamber_y3',
                             'chinstrap_y3', 'humidifier_y3', 'filter_y3']
    df_y3 = data.loc[data['cnt_so_y3'] > 0, resupply_var_y3]
    summary3 = df_y3.groupby([freq_var, group_var]).sum().astype('Int64')
    summary3_count = df_y3[['airviewecn', freq_var, group_var]].groupby([freq_var, group_var]).nunique()
    summary3= pd.merge(summary3_count, summary3, how = 'left', on = [freq_var, group_var])
    summary3 = summary3.rename(columns={'airviewecn':'count_y3'})

    summary3['pap_y3'] = summary3.loc[:, ['cpap_y3', 'bilevel_y3']].sum(axis=1)
    summary3['mask_y3'] = summary3.loc[:, ['ff_mask_y3', 'np_mask_y3', 'on_mask_y3']].sum(axis=1)
    summary3['cushion_y3'] = summary3.loc[:, ['ff_cushion_y3', 'np_cushion_y3', 'on_cushion_y3']].sum(axis=1)

    # fourth year
    resupply_var_y4 = ['airviewecn', freq_var, group_var, 'cpap_y4', 'bilevel_y4', 'ff_mask_y4',
                             'ff_cushion_y4', 'np_mask_y4', 'np_cushion_y4', 'on_mask_y4',
                             'on_cushion_y4', 'headgear_y4', 'tubing_y4', 'waterchamber_y4',
                             'chinstrap_y4', 'humidifier_y4', 'filter_y4']
    df_y4 = data.loc[data['cnt_so_y4'] > 0, resupply_var_y4]
    summary4 = df_y4.groupby([freq_var, group_var]).sum().astype('Int64')
    summary4_count = df_y4[['airviewecn', freq_var, group_var]].groupby([freq_var, group_var]).nunique()
    summary4= pd.merge(summary4_count, summary4, how='left', on=[freq_var, group_var])
    summary4 = summary4.rename(columns={'airviewecn':'count_y4'})

    summary4['pap_y4'] = summary4.loc[:, ['cpap_y4', 'bilevel_y4']].sum(axis=1)
    summary4['mask_y4'] = summary4.loc[:, ['ff_mask_y4', 'np_mask_y4', 'on_mask_y4']].sum(axis=1)
    summary4['cushion_y4'] = summary4.loc[:, ['ff_cushion_y4', 'np_cushion_y4', 'on_cushion_y4']].sum(axis=1)

    # fifth year
    resupply_var_y5 = ['airviewecn', freq_var, group_var, 'cpap_y5', 'bilevel_y5', 'ff_mask_y5',
                             'ff_cushion_y5', 'np_mask_y5', 'np_cushion_y5', 'on_mask_y5',
                             'on_cushion_y5', 'headgear_y5', 'tubing_y5', 'waterchamber_y5',
                             'chinstrap_y5', 'humidifier_y5', 'filter_y5']
    df_y5 = data.loc[data['cnt_so_y5'] > 0, resupply_var_y5]
    summary5 = df_y5.groupby([freq_var, group_var]).sum().astype('Int64')
    summary5_count = df_y5[['airviewecn', freq_var, group_var]].groupby([freq_var, group_var]).nunique()
    summary5= pd.merge(summary5_count, summary5, how='left', on=[freq_var, group_var])
    summary5 = summary5.rename(columns={'airviewecn':'count_y5'})

    summary5['pap_y5'] = summary5.loc[:, ['cpap_y5', 'bilevel_y5']].sum(axis=1)
    summary5['mask_y5'] = summary5.loc[:, ['ff_mask_y5', 'np_mask_y5','on_mask_y5']].sum(axis=1)
    summary5['cushion_y5'] = summary5.loc[:, ['ff_cushion_y5', 'np_cushion_y5', 'on_cushion_y5']].sum(axis=1)

    # after fifth year
    resupply_var_after_5th_year = ['airviewecn', freq_var, group_var, 'cpap_after_5th_year', 'bilevel_after_5th_year',
                                   'ff_mask_after_5th_year', 'ff_cushion_after_5th_year', 'np_mask_after_5th_year',
                                   'np_cushion_after_5th_year', 'on_mask_after_5th_year', 'on_cushion_after_5th_year',
                                   'headgear_after_5th_year', 'tubing_after_5th_year', 'waterchamber_after_5th_year',
                                   'chinstrap_after_5th_year', 'humidifier_after_5th_year', 'filter_after_5th_year']
    df_after_5th_year = data.loc[data['cnt_so_after_5th_year'] > 0, resupply_var_after_5th_year]
    summary6 = df_after_5th_year.groupby([freq_var, group_var]).sum().astype('Int64')
    summary6_count = df_after_5th_year[['airviewecn', freq_var, group_var]].groupby([freq_var, group_var]).nunique()
    summary6 = pd.merge(summary6_count, summary6, how='left', on=[freq_var, group_var])
    summary6 = summary6.rename(columns={'airviewecn':'count_after_5th_year'})

    summary6['pap_after_5th_year'] = summary6.loc[:, ['cpap_after_5th_year', 'bilevel_after_5th_year']].sum(axis=1)
    summary6['mask_after_5th_year'] = summary6.loc[:, ['ff_mask_after_5th_year', 'np_mask_after_5th_year', 'on_mask_after_5th_year']].sum(axis=1)
    summary6['cushion_after_5th_year'] = summary6.loc[:, ['ff_cushion_after_5th_year', 'np_cushion_after_5th_year', 'on_cushion_after_5th_year']].sum(axis=1)

    # merge resupply tables
    summary_resupply = pd.merge(summary1[['compliance','count_y1', 'pap_y1', 'mask_y1', 'cushion_y1',
                                     'headgear_y1', 'tubing_y1', 'waterchamber_y1', 'chinstrap_y1', 'humidifier_y1', 'filter_y1']],
                                summary2[['count_y2', 'pap_y2', 'mask_y2', 'cushion_y2',
                                          'headgear_y2', 'tubing_y2', 'waterchamber_y2', 'chinstrap_y2', 'humidifier_y2', 'filter_y2']],
                                how = 'left', on = [freq_var, group_var])
    summary_resupply = pd.merge(summary_resupply,
                                summary3[['count_y3', 'pap_y3', 'mask_y3', 'cushion_y3',
                                      'headgear_y3', 'tubing_y3', 'waterchamber_y3', 'chinstrap_y3', 'humidifier_y3', 'filter_y3']],
                                how = 'left', on = [freq_var, group_var])
    summary_resupply = pd.merge(summary_resupply,
                                summary4[['count_y4', 'pap_y4', 'mask_y4', 'cushion_y4',
                                      'headgear_y4', 'tubing_y4', 'waterchamber_y4', 'chinstrap_y4', 'humidifier_y4', 'filter_y4']],
                                how = 'left', on = [freq_var, group_var])
    summary_resupply = pd.merge(summary_resupply,
                                summary5[['count_y5', 'pap_y5', 'mask_y5', 'cushion_y5',
                                      'headgear_y5', 'tubing_y5', 'waterchamber_y5', 'chinstrap_y5', 'humidifier_y5', 'filter_y5']],
                                how = 'left', on = [freq_var, group_var])
    summary_resupply = pd.merge(summary_resupply, summary6[['count_after_5th_year', 'pap_after_5th_year', 'mask_after_5th_year', 'cushion_after_5th_year',
                                       'headgear_after_5th_year', 'tubing_after_5th_year', 'waterchamber_after_5th_year', 'chinstrap_after_5th_year',
                                        'humidifier_after_5th_year', 'filter_after_5th_year']],
                             how = 'left', on = [freq_var, group_var])

    # make sure the first three columns are freq_var, group_var, 'compliance'
    summary_resupply.reset_index(inplace=True)

    return summary_resupply

def spilt_columns_into_rows(df):

    """ This function convert 5-year resupply rate into 5 rows each containing one year"""

    cols_all = df.columns.str
    cols_name = ['SetupMonth', 'MaskBrand', 'Compliance', 'PatientCount', 'DeviceCount', 'MaskCount', 'CushionCount', 'HeadgearCount',
                 'TubingCount', 'WaterChamberCount', 'ChinstrapCount', 'HumidifierCount', 'FilterCount']

    col_index = cols_all.contains('y1', case=False)
    # add first three columns of df
    col_index[0:3] = [True, True, True]
    df_1st = df.iloc[:, col_index]
    df_1st.columns = cols_name
    df_1st.insert(1, "Year", 1)

    col_index = cols_all.contains('y2', case=False)
    col_index[0:3] = [True, True, True]
    df_2nd = df.iloc[:, col_index]
    df_2nd.columns = cols_name
    df_2nd.insert(1, "Year", 2)

    col_index = cols_all.contains('y3', case=False)
    col_index[0:3] = [True, True, True]
    df_3rd = df.iloc[:, col_index]
    df_3rd.columns = cols_name
    df_3rd.insert(1, "Year", 3)

    col_index = cols_all.contains('y4', case=False)
    col_index[0:3] = [True, True, True]
    df_4th = df.iloc[:, col_index]
    df_4th.columns = cols_name
    df_4th.insert(1, "Year", 4)

    col_index = cols_all.contains('y5', case=False)
    col_index[0:3] = [True, True, True]
    df_5th = df.iloc[:, col_index]
    df_5th.columns = cols_name
    df_5th.insert(1, "Year", 5)

    df_concat = pd.concat([df_1st, df_2nd, df_3rd, df_4th, df_5th], axis=0)

    return(df_concat)

def get_adherence_rate_kmf(df, surv_days, surv_status, timeline=(np.arange(5)+1)*365):

    """ calculate year1 adherence rate and ongoing attrition rate after year2"""

    kmf = KaplanMeierFitter()
    kmf.fit(df[surv_days], df[surv_status], timeline=timeline)
    surv_table = kmf.survival_function_

    year1_adherence = float(surv_table.loc[[365.0], "KM_estimate"])
    year2_adherence = float(surv_table.loc[[730.0], "KM_estimate"])
    if year1_adherence > 0:
        ongoing_compliance = year2_adherence/year1_adherence
    else:
        ongoing_compliance = float("nan")

    adherence = {"Year1Adherence":round(year1_adherence, 3),
                 "Year2Adherence":round(year2_adherence, 3),
                 "OngoingCompliance":round(ongoing_compliance, 3)}

    return adherence

def get_adherence_rate_by_mask_setup_month(df_selected, count_size=100):

    timeline = (np.arange(5)+1)*365
    adherence_setup_month = []

    for i in range(len(setup_month_list)):
        month_tp = setup_month_list[i]

        for j in mask_brand_list:

            ind = (df_selected['setup_month'] == month_tp) & (df_selected['first_so_mask_brand'] == j) & \
                (df_selected['first_so_device_hcpc'] == 'E0601') & (df_selected['flag_same_mask_brand_first_two_years'] == 1)
            df_tp = df_selected.loc[ind]
            df_tp = df_tp[['survival_days', 'is_dropout']].dropna()

            if df_tp.shape[0] > count_size:
                adherence_tp = get_adherence_rate_kmf(df_tp, 'survival_days', 'is_dropout', timeline=timeline)
                output_tp = [month_tp, j, adherence_tp["Year1Adherence"], adherence_tp["Year2Adherence"],                                                                                                                      adherence_tp["OngoingCompliance"]]
                adherence_setup_month.append(output_tp)

    adherence_setup_month = pd.DataFrame(adherence_setup_month)
    adherence_setup_month.columns = ['SetupMonth', 'MaskBrand', 'Year1Adherence', 'Year2Adherence', 'OngoingCompliance']

    return adherence_setup_month

def get_adherence_rate_manually(df_selected):

    # calculate year1 and year2 adherence rates manually groupby mask brand and setup month
    df_selected['global_av_last_record_date'] = pd.to_datetime(df_selected['global_av_last_record_date']).dt.date
    df_selected['days_setup_last_av'] = (df_selected['global_av_last_record_date'] - df_selected['setup_date']).dt.days + 1

    # convert data type and define new variable
    df_selected['global_av_last_record_date'] = pd.to_datetime(df_selected['global_av_last_record_date']).dt.date
    df_selected['days_setup_last_av'] = (df_selected['global_av_last_record_date'] - df_selected['setup_date']).dt.days + 1

    # count setup in the beginning
    cnt_total = df_selected.groupby(['setup_month', 'first_so_mask_brand']).size().to_frame('cnt_total')

    # count those survive year1 (days_setup_last_av >= 395 & survival_days >= 365)
    # 30 days difference in between is because of drop out criteria
    ind_y1 = (df_selected['days_setup_last_av'] >= 395) & (df_selected['survival_days'] >= 365)
    cnt_surv_y1 = df_selected.loc[ind_y1].groupby(['setup_month', 'first_so_mask_brand']).size().to_frame('cnt_after_y1')

    # count those survive year2 (days_setup_last_av >= 760 & survival_days >= 730)
    # 30 days difference in between is because of drop out criteria
    ind_y2 = (df_selected['days_setup_last_av'] >= 760) & (df_selected['survival_days'] >= 730)
    cnt_surv_y2 = df_selected.loc[ind_y2].groupby(['setup_month', 'first_so_mask_brand']).size().to_frame('cnt_after_y2')

    # merge
    cnt_adherence = cnt_total.join(cnt_surv_y1)
    cnt_adherence = cnt_adherence.join(cnt_surv_y2)
    cnt_adherence['cnt_after_y1'] = cnt_adherence['cnt_after_y1'].fillna(0).astype(float)
    cnt_adherence['cnt_after_y2'] = cnt_adherence['cnt_after_y2'].fillna(0).astype(float)

    # define year1 adherence, year2 adherence, ongoingcompliance
    cnt_adherence['Year1Adherence'] = cnt_adherence['cnt_after_y1']/cnt_adherence['cnt_total']
    cnt_adherence['Year2Adherence'] = cnt_adherence['cnt_after_y2']/cnt_adherence['cnt_total']
    cnt_adherence['OngoingCompliance'] = np.where(cnt_adherence['cnt_after_y1'] > 0,                                                                                                                                                              cnt_adherence['cnt_after_y2']/cnt_adherence['cnt_after_y1'], float("nan"))

    cnt_adherence.reset_index(inplace = True)
    cnt_adherence.drop(['cnt_after_y1', 'cnt_after_y2'], axis=1, inplace = True)
    cnt_adherence.columns = ['SetupMonth', 'MaskBrand', 'Count', 'Year1Adherence', 'Year2Adherence', 'OngoingCompliance']

    return cnt_adherence

def modify_reorder_historical(df_resupply_adherence):

    del df_resupply_adherence['SetupMonth']
    cols = df_resupply_adherence.columns.tolist()
    cols = ['SetupDate'] + ['ResupplyDate']  + cols[:-5] + cols[-3:]
    df_resupply_adherence = df_resupply_adherence[cols]
    df_resupply_adherence[['PatientCount', 'DeviceCount', 'MaskCount', 'CushionCount']] = df_resupply_adherence[['PatientCount', 'DeviceCount', 'MaskCount', 'CushionCount']].astype('Int64')
    df_resupply_adherence[['Compliance']] = df_resupply_adherence[['Compliance']].round(3)
    df_resupply_adherence = df_resupply_adherence.sort_values(['SetupDate', 'ResupplyDate'])

    return df_resupply_adherence


In [ ]:
##-------------------------------------------------------------------------
## 3 Generate naive forecast of resupply and adherence (utils)
##-------------------------------------------------------------------------

def get_weighted_average_setup_month(df, mask_infor, setup_dt_resupply, setup_dt_adherence, year=1, mask_cat='FF Mask'):

    mask_list = list(mask_infor.loc[mask_infor['MaskCategory'] == mask_cat, 'MaskBrandLower'])
    var_list_resupply = ['Compliance', 'PatientCount', 'DeviceCount', 'MaskCount', 'CushionCount', 'HeadgearCount', 'TubingCount',
                         'WaterChamberCount', 'ChinstrapCount', 'HumidifierCount', 'FilterCount']
    var_list_adherence = ['PatientCount', 'Year1Adherence', 'Year2Adherence']

    # resupply
    ind = (df['Year'] == year) & (df['SetupDate'] == setup_dt_resupply) & (df['MaskBrand'].isin(mask_list))
    df_resupply = df.loc[ind, var_list_resupply].dropna()
    df_resupply.loc[:,'DeviceCount':'FilterCount'] = df_resupply.loc[:, 'DeviceCount':'FilterCount'].div(df_resupply['PatientCount'], axis=0)
    patient_count_total = df_resupply['PatientCount'].sum()
    df_resupply['PatientCount_Percentage'] = df_resupply['PatientCount']/patient_count_total
    percentage = df_resupply['PatientCount_Percentage'].values
    s_weighted_average_resupply = df_resupply.apply(lambda x: np.inner(x, percentage), axis=0)
    s_weighted_average_resupply.drop(labels=['PatientCount', 'PatientCount_Percentage'], inplace=True)

    # adherence
    ind = (df['Year'] == year) & (df['SetupDate'] == setup_dt_adherence) & (df['MaskBrand'].isin(mask_list))
    df_adherence = df.loc[ind, var_list_adherence].dropna()
    patient_count_total = df_adherence['PatientCount'].sum()
    df_adherence['PatientCount_Percentage'] = df_adherence['PatientCount']/patient_count_total
    percentage = df_adherence['PatientCount_Percentage'].values
    s_weighted_average_adherence = df_adherence.apply(lambda x: np.inner(x, percentage), axis=0)
    s_weighted_average_adherence.drop(labels=['PatientCount', 'PatientCount_Percentage'], inplace=True)

    # combine
    s_info = pd.Series({'SetupDate': setup_dt_resupply, 'Year': year, 'MaskCategory': mask_cat})
    s_weighted_average = s_info.append(s_weighted_average_resupply)
    s_weighted_average = s_weighted_average.append(s_weighted_average_adherence)
    df_weighted_average = pd.DataFrame(s_weighted_average).transpose()

    return df_weighted_average

def get_weighted_average(df, df_selected, mask_infor):
    # calculate lastest available resupply and adherence rate (year1 and year2) of mask category for naive forecast

    ## global last sale order confirm date
    global_last_so_dt = pd.to_datetime(min(df_selected['global_last_so_cf_dt'])).replace(day=1)
    dt_recent_first_year_resupply = global_last_so_dt + relativedelta(months=-13)
    dt_recent_2nd_year_resupply = global_last_so_dt + relativedelta(months=-25)

    ## global last airview record date
    global_last_av_dt = pd.to_datetime(min(df_selected['global_av_last_record_date'])).replace(day=1)
    # setup month one year ago and two year ago for adherence
    # those two dates are 1 month earlier than corresponding resupply dates because of dropout definition
    # also last sale order date and last airview record date might be different
    dt_recent_first_year_adherence = global_last_av_dt + relativedelta(months=-14)
    dt_recent_2nd_year_adherence = global_last_av_dt + relativedelta(months=-26)

    df_weighted_average = []
    for mask_cat in ['FF Mask', 'Nasal', 'Pillow']:

        df_weighted_average_1 = get_weighted_average_setup_month(df, mask_infor, dt_recent_first_year_resupply,                                                                                              dt_recent_first_year_adherence, 1, mask_cat)
        df_weighted_average_2 = get_weighted_average_setup_month(df, mask_infor, dt_recent_2nd_year_resupply,
                                                                 dt_recent_2nd_year_adherence, 1, mask_cat)
        df_weighted_average_3 = get_weighted_average_setup_month(df, mask_infor, dt_recent_2nd_year_resupply,
                                                                 dt_recent_2nd_year_adherence, 2, mask_cat)
        df_weighted_average_tp = pd.concat([df_weighted_average_1, df_weighted_average_2, df_weighted_average_3])
        df_weighted_average.append(df_weighted_average_tp)

    df_weighted_average = pd.concat(df_weighted_average)
    df_weighted_average['OngoingCompliance'] = df_weighted_average['Year2Adherence']/df_weighted_average['Year1Adherence']

    return df_weighted_average

def get_first_month_with_large_setup_count(df_resupply_adherence, mask_infor, count_size=100):

    df_first_setup_month = []
    mask_brand_list = list(mask_infor['MaskBrandLower'].unique())
    for j in mask_brand_list:
        ind = (df_resupply_adherence['MaskBrand'] == j) & (df_resupply_adherence['PatientCount'] > count_size)
        ind = ind & (df_resupply_adherence['Year'] == 1)
        if (sum(ind) > 0):
            df_mask = df_resupply_adherence.loc[ind, ['SetupDate', 'MaskBrand']].sort_values(['SetupDate']).reset_index(drop=True)
            df_first_setup_month_tp = df_mask.head(1)
            df_first_setup_month.append(df_first_setup_month_tp)
    df_first_setup_month = pd.concat(df_first_setup_month, ignore_index=True)

    return df_first_setup_month

def naive_forecast_setup_within_one_year(df, setup_dt_list_resupply, setup_dt_list_adherence, resupply_count_recent_first_year, resupply_count_recent_2nd_year, adherence_recent):
    """setup date less than one year. Need to forecast lastest year1 and year2 resupply and adherence"""

    len_resupply = len(setup_dt_list_resupply)
    len_adherence = len(setup_dt_list_adherence)

    # list of column names
    list_col_setup = ['SetupDate', 'Year', 'ResupplyDate', 'MaskBrand', 'Compliance', 'PatientCount']
    list_col_adherence = ['Year1Adherence', 'OngoingCompliance', 'Year2Adherence']
    list_col_resupply = ['DeviceCount', 'MaskCount', 'CushionCount', 'HeadgearCount', 'TubingCount',
                        'WaterChamberCount', 'ChinstrapCount', 'HumidifierCount', 'FilterCount']

    ## construct first year resupply for setup date in list_setup_recent_first_year
    # get PatientCount in year1
    ind = (df['SetupDate'].isin(setup_dt_list_resupply)) & (df['Year'] == 1)
    df_ptnt_count_first_year = df.loc[ind, list_col_setup].set_index(['SetupDate', 'Year'])
    # resupply rate of year1
    list_resupply_first_year = resupply_count_recent_first_year.loc[:, 'DeviceCount':'FilterCount'].values.tolist()[0]
    df_resupply_first_year = pd.DataFrame([list_resupply_first_year], columns=list_col_resupply)
    resupply_forecast_first_year = pd.concat([df_resupply_first_year]*len_resupply,ignore_index=True)
    resupply_forecast_first_year['SetupDate'] = setup_dt_list_resupply
    resupply_forecast_first_year['Year'] = 1
    resupply_forecast_first_year.set_index(['SetupDate', 'Year'], inplace=True)

    df_resupply_forecast_first_year = df_ptnt_count_first_year.join(resupply_forecast_first_year)
    # resupply count of year1
    df_resupply_forecast_first_year.loc[:, 'DeviceCount':'FilterCount'] = df_resupply_forecast_first_year.loc[:,
                                           'DeviceCount':'FilterCount'].multiply(df_resupply_forecast_first_year['PatientCount'], axis="index")
    df_resupply_forecast_first_year.loc[:, 'DeviceCount':'FilterCount'] = df_resupply_forecast_first_year.loc[:, 'DeviceCount':'FilterCount'].astype(float).round(0)
    # update year1 resupply count of df
    df.set_index(['SetupDate', 'Year'], inplace=True)
    df.update(df_resupply_forecast_first_year)
    # adherence
    list_adherence_first_year = adherence_recent.values.tolist()[0]
    df_adherence_first_year = pd.DataFrame([list_adherence_first_year], columns=list_col_adherence)
    adherence_forecast_first_year = pd.concat([df_adherence_first_year]*len_adherence, ignore_index=True)
    adherence_forecast_first_year['SetupDate'] = setup_dt_list_adherence
    adherence_forecast_first_year['Year'] = 1
    adherence_forecast_first_year.set_index(['SetupDate', 'Year'], inplace=True)
    # update adherence of df
    df.update(adherence_forecast_first_year)
    df.reset_index(inplace=True)

    ## construct 2nd year resupply for setup date in list_setup_recent_first_year
    # get PatientCount in year2
    ind = (df['SetupDate'].isin(setup_dt_list_resupply)) & (df['Year'] == 1)
    df_ptnt_count_2nd_year = df.loc[ind, (list_col_setup + list_col_adherence)]
    df_ptnt_count_2nd_year['Year'] = 2
    df_ptnt_count_2nd_year['ResupplyDate'] = df_ptnt_count_2nd_year['SetupDate']  + pd.DateOffset(years=1)
    df_ptnt_count_2nd_year['PatientCount'] = df_ptnt_count_2nd_year['PatientCount'] * df_ptnt_count_2nd_year['Year1Adherence']
    df_ptnt_count_2nd_year.set_index(['SetupDate', 'Year'], inplace=True)
    # resupply rate of year2
    list_resupply_2nd_year = resupply_count_recent_2nd_year.loc[:, 'DeviceCount':'FilterCount'].values.tolist()[0]
    df_resupply_2nd_year = pd.DataFrame([list_resupply_2nd_year],columns = list_col_resupply)
    resupply_forecast_2nd_year = pd.concat([df_resupply_2nd_year]*len_resupply, ignore_index=True)
    resupply_forecast_2nd_year['SetupDate'] = setup_dt_list_resupply
    resupply_forecast_2nd_year['Year'] = 2
    resupply_forecast_2nd_year.set_index(['SetupDate', 'Year'], inplace=True)

    df_forecast_2nd_year = df_ptnt_count_2nd_year.join(resupply_forecast_2nd_year)
    # resupply count of year2
    df_forecast_2nd_year.loc[:, 'DeviceCount':'FilterCount'] = df_forecast_2nd_year.loc[:, 'DeviceCount':'FilterCount'].multiply(df_forecast_2nd_year['PatientCount'], axis="index")
    df_forecast_2nd_year.reset_index(inplace=True)
    df_resupply_forecast_first_year.loc[:, 'PatientCount':'FilterCount'] = df_resupply_forecast_first_year.loc[:, 'PatientCount':'FilterCount'].astype(float).round(0)
    # remove and append
    df_history = df.loc[~((df['SetupDate'].isin(setup_dt_list_resupply)) & (df['Year'] == 2))]
    df_update = pd.concat([df_history, df_forecast_2nd_year], ignore_index=True)

    return df_update

def naive_forecast_setup_one_year_earlier(df, setup_dt_list_resupply, setup_dt_list_adherence, resupply_count_recent_first_year, resupply_count_recent_2nd_year, adherence_recent):
    """setup date more than one year but less than two years.
       Need to retain year1 resupply and adherence while forecast year2 resupply and adherence"""
    # usually min(setup_dt_list_resupply) > min(setup_dt_list_adherence) because of late airview refresh and drop out definition
    len_resupply = len(setup_dt_list_resupply)
    len_adherence = len(setup_dt_list_adherence)

    # list of column names
    list_col_setup = ['SetupDate', 'Year', 'ResupplyDate', 'MaskBrand', 'Compliance', 'PatientCount']
    list_col_adherence = ['Year1Adherence', 'OngoingCompliance', 'Year2Adherence']
    list_col_resupply = ['DeviceCount', 'MaskCount', 'CushionCount', 'HeadgearCount', 'TubingCount',
                        'WaterChamberCount', 'ChinstrapCount', 'HumidifierCount', 'FilterCount']

    ### Year = 1
    ## update Year2Adherence (Year1Adherence * OngoingCompliance) for setup date in setup_dt_list_adherence
    list_ongoing_compliance = adherence_recent['OngoingCompliance'].values[0]
    df_ongoing_compliance = pd.DataFrame([list_ongoing_compliance], columns=['OngoingCompliance'])
    df_adherence_first_year = pd.concat([df_ongoing_compliance]*len_adherence, ignore_index=True)
    df_adherence_first_year['SetupDate'] = setup_dt_list_adherence
    df_adherence_first_year['Year'] = 1
    ind = (df['SetupDate'].isin(setup_dt_list_adherence)) & (df['Year']==1)
    df_adherence = pd.merge(df.loc[ind, ['SetupDate', 'Year', 'Year1Adherence']], df_adherence_first_year,
                            how = 'left', on = ['SetupDate', 'Year'])
    df_adherence['Year2Adherence'] = df_adherence['Year1Adherence'] * df_adherence['OngoingCompliance']
    df_adherence.set_index(['SetupDate', 'Year'], inplace=True)
    df.set_index(['SetupDate', 'Year'], inplace=True)
    df.update(df_adherence)

    ### Year = 2
    ## update Year2Adherence for setup date in setup_dt_list_adherence not in setup_dt_list_resupply
    # keep year2 resupply for these setup date
    diff_adherence_resupply = set(setup_dt_list_adherence).difference(set(setup_dt_list_resupply))
    if len(list(diff_adherence_resupply)) > 0:
        df_adherence_new = df_adherence.reset_index().copy()
        ind = (df_adherence_new['SetupDate'].isin(list(diff_adherence_resupply)))
        if sum(ind) > 0:
            df_adherence_new = df_adherence_new.loc[ind]
            df_adherence_new['Year'] = 2
            df_adherence_new.set_index(['SetupDate', 'Year'], inplace=True)
            df.update(df_adherence_new)
    df.reset_index(inplace=True)

    ## construct year2 resupply for setup date in setup_dt_list_resupply
    # get PatientCount in year2
    ind = (df['SetupDate'].isin(setup_dt_list_resupply)) & (df['Year']==1)
    df_ptnt_count_2nd_year = df.loc[ind, (list_col_setup + list_col_adherence)]
    df_ptnt_count_2nd_year['Year'] = 2
    df_ptnt_count_2nd_year['ResupplyDate'] = df_ptnt_count_2nd_year['SetupDate']  + pd.DateOffset(years=1)
    df_ptnt_count_2nd_year['PatientCount'] = df_ptnt_count_2nd_year['PatientCount'] * df_ptnt_count_2nd_year['Year1Adherence']
    df_ptnt_count_2nd_year.set_index(['SetupDate', 'Year'], inplace=True)

    list_resupply_2nd_year = resupply_count_recent_2nd_year.loc[:, 'DeviceCount':'FilterCount'].values.tolist()[0]
    df_resupply_2nd_year = pd.DataFrame([list_resupply_2nd_year], columns=list_col_resupply)
    resupply_forecast_2nd_year = pd.concat([df_resupply_2nd_year]*len_resupply, ignore_index=True)
    resupply_forecast_2nd_year['SetupDate'] = setup_dt_list_resupply
    resupply_forecast_2nd_year['Year'] = 2
    resupply_forecast_2nd_year.set_index(['SetupDate', 'Year'], inplace=True)

    df_forecast_2nd_year = df_ptnt_count_2nd_year.join(resupply_forecast_2nd_year)
    df_forecast_2nd_year.loc[:, 'DeviceCount':'FilterCount'] = df_forecast_2nd_year.loc[:, 'DeviceCount':'FilterCount'].multiply(df_forecast_2nd_year['PatientCount'], axis="index")
    df_forecast_2nd_year.loc[:, 'DeviceCount':'FilterCount'] = df_forecast_2nd_year.loc[:, 'DeviceCount':'FilterCount'].astype(float).round(0)
    df_forecast_2nd_year['PatientCount'] = df_forecast_2nd_year['PatientCount'].astype(float).round(0)
    df_forecast_2nd_year.reset_index(inplace=True)

    # remove and append
    df_history = df.loc[~((df['SetupDate'].isin(setup_dt_list_resupply)) & (df['Year'] == 2))]
    df_update = pd.concat([df_history, df_forecast_2nd_year], ignore_index=True)

    return df_update

def naive_forecast_resupply_adherence(mask_brand, df_resupply_adherence, df_selected, df_weighted_average, mask_infor, flag_ptnt_count_100more, flag_mask_brand, flag_mix):

    # this function provide naive forecast for mask brand
    df_mask = df_resupply_adherence.loc[df_resupply_adherence['MaskBrand'] == mask_brand].sort_values(['SetupDate','Year'])

    ## global last airview record date
    global_last_av_dt = pd.to_datetime(min(df_selected['global_av_last_record_date'])).replace(day=1)
    dt_recent_first_year_adherence = global_last_av_dt + relativedelta(months=-14)
    dt_recent_2nd_year_adherence = global_last_av_dt + relativedelta(months=-26)
    # 'SetupDate' list in recent 2 years (with year1 adherence but not year2 adherence)
    list_setup_recent_2nd_year_adherence = [global_last_av_dt + relativedelta(months = -(i+14)) for i in range(12)]
    ## global last sale order confirm date
    global_last_so_dt = pd.to_datetime(min(df_selected['global_last_so_cf_dt'])).replace(day=1)
    dt_recent_first_year_resupply = global_last_so_dt + relativedelta(months=-13)
    dt_recent_2nd_year_resupply = global_last_so_dt + relativedelta(months=-25)
    dt_last_complete_month = global_last_so_dt + relativedelta(months=-1)
    # 'SetupDate' list in recent 1 years (without year1 adherence)
    list_setup_recent_first_year_adherence = [i for i in df_resupply_adherence['SetupDate'].unique()
                                              if (i > dt_recent_first_year_adherence) & (i <= dt_last_complete_month)]
    # 'SetupDate' list in recent 2 years (with year1 resupply but not year2 resupply)
    list_setup_recent_2nd_year_resupply = [global_last_so_dt + relativedelta(months = -(i+13)) for i in range(12)]
    # 'SetupDate' list in recent 1 years (without year1 resupply)
    list_setup_recent_first_year_resupply = [i for i in df_resupply_adherence['SetupDate'].unique()
                                             if (i > dt_recent_first_year_resupply) & (i <= dt_last_complete_month)]

    # list of column names
    list_col_setup = ['SetupDate', 'Year', 'ResupplyDate', 'MaskBrand', 'Compliance', 'PatientCount']
    list_col_adherence = ['Year1Adherence', 'OngoingCompliance', 'Year2Adherence']
    list_col_resupply = ['DeviceCount', 'MaskCount', 'CushionCount', 'HeadgearCount', 'TubingCount',
                        'WaterChamberCount', 'ChinstrapCount', 'HumidifierCount', 'FilterCount']

    ## latest two years resupply and adherence
    if flag_mask_brand & flag_ptnt_count_100more:
        # mask brand (for mask brand on market more than 2 years)
        # lastest first year resupply rate of this mask brand
        resupply_count_recent_first_year = df_mask.loc[(df_mask['SetupDate'] == dt_recent_first_year_resupply) & (df_mask['Year'] == 1)]
        resupply_count_recent_first_year.loc[:, 'DeviceCount':'FilterCount'] = resupply_count_recent_first_year.loc[:,                                                                                                                             'DeviceCount':'FilterCount'].div(resupply_count_recent_first_year['PatientCount'], axis=0)

        # lastest second year resupply rate of this mask brand
        resupply_count_recent_2nd_year = df_mask.loc[(df_mask['SetupDate'] == dt_recent_2nd_year_resupply) & (df_mask['Year'] == 2)]
        resupply_count_recent_2nd_year.loc[:, 'DeviceCount':'FilterCount'] = resupply_count_recent_2nd_year.loc[:,                                                                                                                               'DeviceCount':'FilterCount'].div(resupply_count_recent_2nd_year['PatientCount'], axis=0)

        # latest two years adherence
        adherence_recent_first_year = df_mask.loc[(df_mask['SetupDate'] == dt_recent_first_year_adherence) & (df_mask['Year'] == 1), list_col_adherence]
        adherence_recent_2nd_year = df_mask.loc[(df_mask['SetupDate'] == dt_recent_2nd_year_adherence) & (df_mask['Year'] == 2), list_col_adherence]
        list_adherence = list(adherence_recent_first_year['Year1Adherence']) + list(adherence_recent_2nd_year['OngoingCompliance'])
        adherence_recent = pd.DataFrame([list_adherence], columns=['Year1Adherence','OngoingCompliance'])

    elif flag_mix & flag_ptnt_count_100more:

        # mix of mask brand and catogory (for mask brand on the market more than 1 year but less than 2 years)
        # lastest first year resupply rate of this mask brand
        resupply_count_recent_first_year = df_mask.loc[(df_mask['SetupDate'] == dt_recent_first_year_resupply) & (df_mask['Year'] == 1)]
        resupply_count_recent_first_year.loc[:, 'DeviceCount':'FilterCount'] = resupply_count_recent_first_year.loc[:,                                                                                                                             'DeviceCount':'FilterCount'].div(resupply_count_recent_first_year['PatientCount'], axis=0)

        # lastest second year resupply rate of this mask category
        mask_cat = mask_infor.loc[mask_infor['MaskBrandLower'] == mask_brand, 'MaskCategory'].values[0]
        ind = (df_weighted_average['MaskCategory'] == mask_cat) & (df_weighted_average['Year'] == 2)\
            & (df_weighted_average['SetupDate'] == dt_recent_2nd_year_resupply)
        resupply_count_recent_2nd_year = df_weighted_average.loc[ind]

        # latest two years adherence
        adherence_recent_first_year = df_mask.loc[(df_mask['SetupDate'] == dt_recent_first_year_adherence) & (df_mask['Year'] == 1), list_col_adherence]
        ind = (df_weighted_average['MaskCategory'] == mask_cat) & (df_weighted_average['Year'] == 1)\
            & (df_weighted_average['SetupDate'] == dt_recent_2nd_year_resupply)
        list_adherence = list(adherence_recent_first_year['Year1Adherence']) + list(df_weighted_average.loc[ind,'OngoingCompliance'])
        adherence_recent = pd.DataFrame([list_adherence], columns=['Year1Adherence','OngoingCompliance'])

    else:

        # mask category (for mask brand on market less than 1 year, or with less than 100 setup)
        mask_cat = mask_infor.loc[mask_infor['MaskBrandLower'] == mask_brand, 'MaskCategory'].values[0]
        # lastest first year resupply rate of this mask category
        ind = (df_weighted_average['MaskCategory'] == mask_cat) & (df_weighted_average['Year'] == 1)\
            & (df_weighted_average['SetupDate'] == dt_recent_first_year_resupply)
        resupply_count_recent_first_year = df_weighted_average.loc[ind]

        # lastest second year resupply rate of this mask category
        ind = (df_weighted_average['MaskCategory'] == mask_cat) & (df_weighted_average['Year'] == 2)\
            & (df_weighted_average['SetupDate'] == dt_recent_2nd_year_resupply)
        resupply_count_recent_2nd_year = df_weighted_average.loc[ind]

        ## latest two years adherence
        ind = (df_weighted_average['MaskCategory'] == mask_cat) & (df_weighted_average['Year'] == 1)\
            & (df_weighted_average['SetupDate'] == dt_recent_2nd_year_resupply)
        list_adherence = list(resupply_count_recent_first_year['Year1Adherence']) + list(df_weighted_average.loc[ind,'OngoingCompliance'])
        adherence_recent = pd.DataFrame([list_adherence], columns=['Year1Adherence','OngoingCompliance'])

    # forecast year2 adherence using ongoing compliance times year1 adherence
    adherence_recent['Year2Adherence'] = adherence_recent['Year1Adherence'] * adherence_recent['OngoingCompliance']

    ## naive forecast
    df_naive_forecast = naive_forecast_setup_within_one_year(df_mask, list_setup_recent_first_year_resupply, list_setup_recent_first_year_adherence,
                                                             resupply_count_recent_first_year, resupply_count_recent_2nd_year, adherence_recent)
    df_naive_forecast = naive_forecast_setup_one_year_earlier(df_naive_forecast, list_setup_recent_2nd_year_resupply, list_setup_recent_2nd_year_adherence,
                                                              resupply_count_recent_first_year, resupply_count_recent_2nd_year, adherence_recent)

    df_naive_forecast = df_naive_forecast.dropna()
    df_naive_forecast.loc[:, 'PatientCount':'FilterCount'] = df_naive_forecast.loc[:, 'PatientCount':'FilterCount'].astype('int64')

    return(df_naive_forecast)

def get_naive_forecast(df_resupply_adherence, df_selected, df_weighted_average, mask_infor, df_first_setup_month):

    ## global last airview record date
    global_last_av_dt = pd.to_datetime(min(df_selected['global_av_last_record_date'])).replace(day=1)
    dt_recent_first_year_adherence = global_last_av_dt + relativedelta(months=-14)
    dt_recent_2nd_year_adherence = global_last_av_dt + relativedelta(months=-26)
    ## global last sale order confirm date
    global_last_so_dt = pd.to_datetime(min(df_selected['global_last_so_cf_dt'])).replace(day=1)
    dt_recent_first_year_resupply = global_last_so_dt + relativedelta(months=-13)
    dt_recent_2nd_year_resupply = global_last_so_dt + relativedelta(months=-25)

    # get mask list on market more than 2 years and less than 2 years seperately
    ind = (df_first_setup_month['SetupDate'] <= dt_recent_2nd_year_adherence)
    mask_brand_on_market_2years = list(df_first_setup_month.loc[ind, 'MaskBrand'])
    ind = (df_first_setup_month['SetupDate'] > dt_recent_2nd_year_adherence)\
        & (df_first_setup_month['SetupDate'] <= dt_recent_first_year_adherence)
    mask_brand_on_market_1year_more = list(df_first_setup_month.loc[ind, 'MaskBrand'])
    ind = (df_first_setup_month['SetupDate'] > dt_recent_first_year_adherence)
    mask_brand_on_market_1year_less = list(df_first_setup_month.loc[ind, 'MaskBrand'])

    # mask brand on market more than 2 years
    df_forecast_on_market_2years = []
    for mask_brand in mask_brand_on_market_2years:
        print(mask_brand)
        flag_mask_brand = True
        flag_mix = False
        ind = (df_resupply_adherence['MaskBrand'] == mask_brand) & (df_resupply_adherence['SetupDate'] == dt_recent_first_year_adherence)\
            & (df_resupply_adherence['Year'] == 1)
        flag_ptnt_count_100more = (df_resupply_adherence.loc[ind,'PatientCount'].values[0] >= 100)

        df_mask_forecast = naive_forecast_resupply_adherence(mask_brand, df_resupply_adherence, df_selected, df_weighted_average, mask_infor, flag_ptnt_count_100more, flag_mask_brand, flag_mix)
        df_forecast_on_market_2years.append(df_mask_forecast)
    df_forecast_on_market_2years = pd.concat(df_forecast_on_market_2years)

    # mask brand on market more than 1 year but less than 2 years
    df_forecast_on_market_1year_more = []
    for mask_brand in mask_brand_on_market_1year_more:
        print(mask_brand)
        flag_mask_brand = False
        flag_mix = True
        ind = (df_resupply_adherence['MaskBrand'] == mask_brand) & (df_resupply_adherence['SetupDate'] == dt_recent_first_year_adherence)\
            & (df_resupply_adherence['Year'] == 1)
        flag_ptnt_count_100more = (df_resupply_adherence.loc[ind,'PatientCount'].values[0] >= 100)

        df_mask_forecast = naive_forecast_resupply_adherence(mask_brand, df_resupply_adherence, df_selected, df_weighted_average, mask_infor, flag_ptnt_count_100more, flag_mask_brand, flag_mix)
        df_forecast_on_market_1year_more.append(df_mask_forecast)
    df_forecast_on_market_1year_more = pd.concat(df_forecast_on_market_1year_more)

    # mask brand on market less than 1 year
    df_forecast_on_market_1year_less = []
    for mask_brand in mask_brand_on_market_1year_less:
        print(mask_brand)
        flag_mask_brand = False
        flag_mix = False
        flag_ptnt_count_100more = True
        df_mask_forecast = naive_forecast_resupply_adherence(mask_brand, df_resupply_adherence, df_selected, df_weighted_average, mask_infor, flag_ptnt_count_100more, flag_mask_brand, flag_mix)
        df_forecast_on_market_1year_less.append(df_mask_forecast)
    df_forecast_on_market_1year_less = pd.concat(df_forecast_on_market_1year_less)

    # merge
    df_resupply_adherence_forecast = df_forecast_on_market_2years.append(df_forecast_on_market_1year_more)
    df_resupply_adherence_forecast = df_resupply_adherence_forecast.append(df_forecast_on_market_1year_less)

    ## round compliance, Year1Adherence, Year1Adherence, OngoingCompliance to three decimal places
    df_resupply_adherence_forecast[['Compliance', 'Year1Adherence', 'Year2Adherence', 'OngoingCompliance']] = df_resupply_adherence_forecast[['Compliance', 'Year1Adherence', 'Year2Adherence', 'OngoingCompliance']].round(3)

    return df_resupply_adherence_forecast

def modify_reorder_forecast(df_resupply_adherence_forecast, mask_infor):

    # modify mask brand name to match with finance data
    df_resupply_adherence_forecast.rename(columns={'MaskBrand': 'MaskBrandLower'}, inplace=True)
    df_resupply_adherence_forecast = pd.merge(df_resupply_adherence_forecast,
                    mask_infor[['MaskBrandLower', 'MaskBrand', 'MaskCategory', 'MaskManuf']], how='left', on='MaskBrandLower')
    # drop MaskBrandLower
    df_resupply_adherence_forecast.drop(['MaskBrandLower'], axis=1, inplace=True)

    # reorder columns
    cols = df_resupply_adherence_forecast.columns.tolist()
    cols = ['MaskBrand'] + cols[:-3] + cols[-2:]
    df_resupply_adherence_forecast = df_resupply_adherence_forecast[cols]

    return df_resupply_adherence_forecast


In [ ]:
##-------------------------------------------------------------------------
## Global utils
##-------------------------------------------------------------------------

def move_delete_folder(bucket, folder_src, folder_target, delete_target = True):
    # move files in folder_src to folder_target and delete;
    # get key of object in source/target folders
    s3_client = boto3.client('s3')
    objects_src = s3_client.list_objects_v2(Bucket=bucket, Prefix=folder_src)
    objects_target = s3_client.list_objects_v2(Bucket=bucket, Prefix=folder_target)

    # get count of keys in folders (folder not exists: 0; empry folder: 1)
    cnt_key_src = objects_src['KeyCount']
    cnt_key_target = objects_target['KeyCount']

    # remove files in target folder
    if delete_target & (cnt_key_target > 0):
        for obj in objects_target['Contents']:
            print(obj['Key'])
            s3_client.delete_object(Bucket=bucket, Key=obj['Key'])

    # copy files to target folder and then remove them from source folder
    if cnt_key_src > 0:
        for obj in objects_src['Contents']:
            print(obj['Key'])

            # copy object to target folder from source folder
            s3_client.copy_object(
                CopySource = {'Bucket': bucket, 'Key': obj['Key']},
                Bucket = bucket,
                Key = folder_target + obj['Key'][len(folder_src):]
            )

            # delete objects in source folder
            s3_client.delete_object(Bucket=bucket, Key=obj['Key'])

def delete_folder(bucket, folder_src):
    # move files in folder_src to folder_target and delete;
    # get key of object in source/target folders
    s3_client = boto3.client('s3')
    objects_src = s3_client.list_objects_v2(Bucket=bucket, Prefix=folder_src)

    # get count of keys in folders (folder not exists: 0; empry folder: 1)
    cnt_key_src = objects_src['KeyCount']

    # remove files from source folder
    if cnt_key_src > 0:
        for obj in objects_src['Contents']:
            print(obj['Key'])
            # delete objects in source folder
            s3_client.delete_object(Bucket=bucket, Key=obj['Key'])


# 0 Run Athena query to get patient level resupply, usage and survival data

In [ ]:
##-------------------------------------------------------------------------
## 0 Run Athena query to get patient level resupply, usage and survival data
## The query takes around 2 minutes given the current dataset
##-------------------------------------------------------------------------
logging.info("Athen query starts")
athena_query_src = 'ltv_ljj_2021/athena_query_output/latest'
athena_query_target = 'ltv_ljj_2021/athena_query_output/backup'
# # with open('bt_resupply_final_202112.sql', 'r') as f:
# #     query = f.read()

move_delete_folder(bucket, athena_query_src, athena_query_target)

# Athena execution
athena_client.start_query_execution(
    QueryString=bt_resupply_final_202112,
    ResultConfiguration={
        'OutputLocation': "s3://{}/{}".format(bucket, athena_query_src),
    }
)

ltv_ljj_2021/athena_query_output/backup/9bc6c58b-bb27-4186-be58-27eecfa86f2f.csv
ltv_ljj_2021/athena_query_output/backup/9bc6c58b-bb27-4186-be58-27eecfa86f2f.csv.metadata


{'QueryExecutionId': '7df6d361-783b-437c-94f5-561c116a35e2',
 'ResponseMetadata': {'RequestId': '70e2c882-5a47-4f12-a85e-3de0b17c5cb8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 19 Jan 2022 08:39:25 GMT',
   'x-amzn-requestid': '70e2c882-5a47-4f12-a85e-3de0b17c5cb8',
   'content-length': '59',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

In [ ]:
# sleep for a few minutes to wait athena query to be saved in s3
logging.info("Athen query is done")
sleep(3 * 60)

In [ ]:
##------------------------------------------------------------------
## 1 Load and process resupply data and mask brand information data
##------------------------------------------------------------------
logging.info("Raw data loading and precessing starts")
# load resupply data (paitent level)
resupply_folder_src = 'ltv_ljj_2021/athena_query_output/latest'
df_hist = load_csv_from_s3(bucket, resupply_folder_src)

# load mask brand information data
mask_folder_src = 'ltv_ljj_2021/analysis_output/mask_information/latest'
mask_infor = load_csv_from_s3(bucket, mask_folder_src)

In [ ]:
df_hist.head()

,airviewecn,first_so_cr_dt,first_so_cf_dt,first_so_order_type,flag_first_so_device,flag_first_so_mask,first_so_device_hcpc,first_so_mask_hcpc,first_so_mask_category,first_so_mask_manuf,...,avg_daily_use_q2,avg_daily_use_q4,days_wuse_90days,days_wuse_q2,days_wuse_q4,days_wuse_4hrs_90days,days_wuse_4hrs_q2,days_wuse_4hrs_q4,global_av_last_record_date,days_diff_setup_first_so_cf
0,0004e696-3e5a-4a39-a76f-836e931edf7f,2021-06-24,2021-08-06,new setup,1,1,E0601,A7030,full face,resmed,...,NaN,NaN,59.0,NaN,NaN,10.0,NaN,NaN,2021-12-08,29
1,00100a82-4ae5-45a4-b293-d129819a9793,2020-12-14,2021-01-05,new setup,1,1,E0601,A7034,<a7034>,unmapped,...,0.000,NaN,17.0,0.0,NaN,14.0,0.0,NaN,2021-12-08,5
2,0012b5b6-e849-4c66-bd75-5bfb4ff43e9e,2020-06-26,2020-07-02,new setup,1,1,E0601,A7030,full face,resmed,...,8.102,7.672,89.0,90.0,90.0,89.0,90.0,90.0,2021-12-08,2
3,001c9c52-16cf-4ae2-8562-7bba77f86753,2020-02-10,2020-02-14,new setup,1,1,E0601,A7034,nasal,resmed,...,3.080,2.267,77.0,58.0,46.0,67.0,41.0,31.0,2021-12-08,1
4,002ecb6b-1c36-4b0e-8e66-7613b976b086,2016-07-20,2016-07-20,new setup,1,1,E0601,A7034,nasal,philips respironics,...,7.389,7.964,89.0,89.0,90.0,83.0,86.0,89.0,2021-12-08,0


In [ ]:
# process df_hist (define new columns and cohort selection)
df_selected = process_resupply_data(df_hist)

# delete df_hist and clear memory
lst = [df_hist]
del df_hist
del lst
import gc
gc.collect()

3712

In [ ]:
df_selected.columns.values

array(['airviewecn', 'first_so_cr_dt', 'first_so_cf_dt',
       'first_so_order_type', 'flag_first_so_device',
       'flag_first_so_mask', 'first_so_device_hcpc', 'first_so_mask_hcpc',
       'first_so_mask_category', 'first_so_mask_manuf',
       'first_so_mask_brand', 'ecn_so', 'cpap_q1', 'bilevel_q1',
       'ff_mask_q1', 'ff_cushion_q1', 'np_mask_q1', 'np_cushion_q1',
       'on_mask_q1', 'on_cushion_q1', 'headgear_q1', 'tubing_q1',
       'waterchamber_q1', 'chinsrap_q1', 'humidifier_q1', 'filter_q1',
       'cnt_so_q1', 'cpap_q2', 'bilevel_q2', 'ff_mask_q2',
       'ff_cushion_q2', 'np_mask_q2', 'np_cushion_q2', 'on_mask_q2',
       'on_cushion_q2', 'headgear_q2', 'tubing_q2', 'waterchamber_q2',
       'chinsrap_q2', 'humidifier_q2', 'filter_q2', 'cnt_so_q2',
       'cpap_y1', 'bilevel_y1', 'ff_mask_y1', 'ff_cushion_y1',
       'np_mask_y1', 'np_cushion_y1', 'on_mask_y1', 'on_cushion_y1',
       'headgear_y1', 'tubing_y1', 'waterchamber_y1', 'chinsrap_y1',
       'humidifier_y1'

In [ ]:
# move mask information data from latest to backup
mask_folder_target = 'ltv_ljj_2021/analysis_output/mask_information/backup'
move_delete_folder(bucket, mask_folder_src, mask_folder_target)

ltv_ljj_2021/analysis_output/mask_information/backup/mask_brand.csv
ltv_ljj_2021/analysis_output/mask_information/latest/mask_brand.csv


In [ ]:
# get last complete setup month (eg from 1st day to last day of a month)
global_last_so_dt = pd.to_datetime(min(df_selected['global_last_so_cf_dt'])).replace(day=1)
print(global_last_so_dt)
dt_last_complete_month = global_last_so_dt + relativedelta(months=-1)
setup_month_list = df_selected.setup_month.unique()
setup_month_list = [i for i in setup_month_list if i.to_timestamp() <= dt_last_complete_month]

2022-01-01 00:00:00


In [ ]:
new_mask_brand_infor = find_new_mask_brand(df_selected, mask_infor)

In [ ]:
if new_mask_brand_infor.shape[0] > 0:
    mask_infor = add_new_mask_brand(mask_infor, new_mask_brand_infor)

In [ ]:
mask_infor.sort_values('MaskManuf')

,MaskBrand,MaskBrandLower,MaskManuf,MaskCategory,LaunchMonth
23,Eson,eson,Fisher&Paykel,Nasal,2015-03
21,Brevida,brevida,Fisher&Paykel,Pillow,2017-03
16,Eson 2,eson 2,Fisher&Paykel,Nasal,2016-12
14,Simplus,simplus,Fisher&Paykel,FF Mask,2015-01
13,Dreamwear Gel Pillow,dreamwear gel pillow,Philips,Pillow,2017-08
22,Dreamwisp,dreamwisp,Philips,Nasal,2019-05
17,Wisp,wisp,Philips,Nasal,2015-01
15,Amara View,amara view,Philips,FF Mask,2015-07
11,Dreamwear Full Face,dreamwear full face,Philips,FF Mask,2018-03
12,Dreamwear Nasal,dreamwear nasal,Philips,Nasal,2015-09


In [ ]:
# columns to be deleted from df_selected
var_del = ['ecn', 'ecn_so']
df_selected.drop(var_del, axis=1, inplace=True)

# move patient resupply information data from latest to backup for other parts of project
df_ptnt_folder_src = 'ltv_ljj_2021/analysis_output/resupply_patient_level/latest'
df_ptnt_folder_target = 'ltv_ljj_2021/analysis_output/resupply_patient_level/backup'
move_delete_folder(bucket, df_ptnt_folder_src, df_ptnt_folder_target)

# write patient resupply information to s3 in csv
# df_resupply_ptnt_dir = 's3://{}/{}/{}'.format(bucket, 'ltv_ljj_2021/analysis_output/resupply_patient_level/latest',
#                                               'df_resupply_ptnt_level.csv')
# df_selected.to_csv(df_resupply_ptnt_dir, index=False)

# write patient resupply information to s3 in parquet
path = 's3://{}/{}/'.format(bucket, 'ltv_ljj_2021/analysis_output/resupply_patient_level/latest')
wr.s3.to_parquet(df_selected, path=path, dataset=True, mode="overwrite", partition_cols=["setup_month"])

ltv_ljj_2021/analysis_output/resupply_patient_level/backup/
ltv_ljj_2021/analysis_output/resupply_patient_level/backup/df_resupply_adherence_for_Hannah.csv
ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-01/9bc58ef64f284a1db4eca9742484f62c.snappy.parquet
ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-02/9bc58ef64f284a1db4eca9742484f62c.snappy.parquet
ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-03/9bc58ef64f284a1db4eca9742484f62c.snappy.parquet
ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-04/9bc58ef64f284a1db4eca9742484f62c.snappy.parquet
ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-05/9bc58ef64f284a1db4eca9742484f62c.snappy.parquet
ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-06/9bc58ef64f284a1db4eca9742484f62c.snappy.parquet
ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-07/9bc

{'paths': ['s3://project-ltv-data-analyst-bucket-prd/ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-01/978b0ccd3f144f0e8f91ffbff8284a12.snappy.parquet',
  's3://project-ltv-data-analyst-bucket-prd/ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-02/978b0ccd3f144f0e8f91ffbff8284a12.snappy.parquet',
  's3://project-ltv-data-analyst-bucket-prd/ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-03/978b0ccd3f144f0e8f91ffbff8284a12.snappy.parquet',
  's3://project-ltv-data-analyst-bucket-prd/ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-04/978b0ccd3f144f0e8f91ffbff8284a12.snappy.parquet',
  's3://project-ltv-data-analyst-bucket-prd/ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-05/978b0ccd3f144f0e8f91ffbff8284a12.snappy.parquet',
  's3://project-ltv-data-analyst-bucket-prd/ltv_ljj_2021/analysis_output/resupply_patient_level/latest/setup_month=2015-06/978

In [ ]:
logging.info("Raw data loading and precessing is done")

# 2 Calculate resupply and adherence by mask brand and setup month

In [ ]:
##------------------------------------------------------------------
## 2 Calculate resupply and adherence by mask brand and setup month
##------------------------------------------------------------------
logging.info("Historical resupply and adherence by mask brand and setup month starts")
# calculate yearly resupply count by setup mask and setup month for mask brand in mask_infor;it takes around 1 minute to run
mask_brand_list = list(mask_infor['MaskBrandLower'])
ind_1 = (df_selected['setup_month'].isin(setup_month_list)) & (df_selected['first_so_mask_brand'].isin(mask_brand_list))
ind_2 = (df_selected['first_so_device_hcpc'] == 'E0601') & (df_selected['flag_same_mask_brand_first_two_years'] == 1)
ind = ind_1 & ind_2
resupply_setup_month = count_resupply_yearly(df_selected.loc[ind], freq_var='setup_month', group_var='first_so_mask_brand')
resupply_setup_month

,setup_month,first_so_mask_brand,compliance,count_y1,pap_y1,mask_y1,cushion_y1,headgear_y1,tubing_y1,waterchamber_y1,...,count_after_5th_year,pap_after_5th_year,mask_after_5th_year,cushion_after_5th_year,headgear_after_5th_year,tubing_after_5th_year,waterchamber_after_5th_year,chinsrap_after_5th_year,humidifier_after_5th_year,filter_after_5th_year
0,2015-01,airfit f10,0.816667,60,63.0,119.0,176.0,100,121,27,...,21.0,3.0,44.0,61.0,31,36,19,0,3,117
1,2015-01,airfit n10,0.736842,19,19.0,37.0,109.0,32,43,7,...,8.0,2.0,25.0,113.0,14,24,14,1,2,136
2,2015-01,airfit p10,0.890909,55,57.0,109.0,333.0,98,103,23,...,25.0,6.0,94.0,403.0,68,71,39,11,6,379
3,2015-01,eson,1.000000,8,9.0,15.0,20.0,14,15,5,...,2.0,0.0,9.0,53.0,8,10,5,0,0,54
4,2015-01,mirage fx,0.750000,12,12.0,24.0,90.0,20,25,5,...,4.0,0.0,13.0,56.0,9,13,9,1,0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716,2021-12,mirage quattro,9.000000,11,11.0,11.0,14.0,11,11,2,...,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1717,2021-12,quattro fx,9.000000,3,3.0,4.0,2.0,2,3,0,...,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1718,2021-12,simplus,9.000000,34,34.0,34.0,37.0,31,33,4,...,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1719,2021-12,swift fx,9.000000,10,10.0,10.0,33.0,11,9,3,...,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [ ]:
# add mask launch month and save it to s3
if new_mask_brand_infor.shape[0] > 0:
    mask_infor = add_mask_launch_month(resupply_setup_month, mask_infor, count_size=10)
mask_infor_dir = 's3://{}/{}/{}'.format(bucket, 'ltv_ljj_2021/analysis_output/mask_information/latest', 'mask_brand.csv')
mask_infor[['MaskBrand','MaskBrandLower','MaskManuf','MaskCategory','LaunchMonth']].to_csv(mask_infor_dir, index=False)

In [ ]:
# rearrange the resupply_setup_month from columns to rows (one setup month corresponds to 5 rows)
resupply_formatted = spilt_columns_into_rows(resupply_setup_month).dropna()

# add resupply date
def add_resupply_date(resupply_formatted):
    # resupply date = setup date + resupply year

    resupply_formatted['SetupDate'] = resupply_formatted['SetupMonth'].dt.to_timestamp(freq='D')
    resupply_formatted['ResupplyMonth'] = resupply_formatted['SetupMonth'] + (resupply_formatted['Year'] - 1) * 12
    resupply_formatted['ResupplyDate'] = resupply_formatted['ResupplyMonth'].dt.to_timestamp(freq='D')
    del resupply_formatted['ResupplyMonth']

    return resupply_formatted

resupply_formatted = add_resupply_date(resupply_formatted)

In [ ]:
resupply_formatted.head()

,SetupMonth,Year,MaskBrand,Compliance,PatientCount,DeviceCount,MaskCount,CushionCount,HeadgearCount,TubingCount,WaterChamberCount,ChinstrapCount,HumidifierCount,FilterCount,SetupDate,ResupplyDate
0,2015-01,1,airfit f10,0.816667,60.0,63.0,119.0,176.0,100,121,27,1,59,421,2015-01-01,2015-01-01
1,2015-01,1,airfit n10,0.736842,19.0,19.0,37.0,109.0,32,43,7,1,18,174,2015-01-01,2015-01-01
2,2015-01,1,airfit p10,0.890909,55.0,57.0,109.0,333.0,98,103,23,27,55,446,2015-01-01,2015-01-01
3,2015-01,1,eson,1.000000,8.0,9.0,15.0,20.0,14,15,5,4,8,37,2015-01-01,2015-01-01
4,2015-01,1,mirage fx,0.750000,12.0,12.0,24.0,90.0,20,25,5,4,12,117,2015-01-01,2015-01-01


In [ ]:
# compute adherence manually by setup month and mask/cushion group;
adherence_setup_month = get_adherence_rate_manually(df_selected)
adherence_setup_month[adherence_setup_month['Count'] > 100]

,SetupMonth,MaskBrand,Count,Year1Adherence,Year2Adherence,OngoingCompliance
20,2015-01,unmapped,215,0.679070,0.530233,0.780822
26,2015-02,airfit p10,103,0.757282,0.572816,0.756410
49,2015-02,unmapped,201,0.666667,0.562189,0.843284
51,2015-02,zest q,116,0.750000,0.594828,0.793103
53,2015-03,airfit f10,105,0.638095,0.495238,0.776119
...,...,...,...,...,...,...
4285,2021-12,airtouch f20,192,0.000000,0.000000,NaN
4294,2021-12,dreamwear full face,149,0.000000,0.000000,NaN
4296,2021-12,dreamwear nasal,204,0.000000,0.000000,NaN
4311,2021-12,unmapped,473,0.000000,0.000000,NaN


In [ ]:
# combine resupply and adherence
df_resupply_adherence = pd.merge(resupply_formatted, adherence_setup_month[['SetupMonth','MaskBrand', 'Year1Adherence', 'Year2Adherence', 'OngoingCompliance']], how='left', on=['SetupMonth', 'MaskBrand'])

# modify column types, add columns and reorder columns
df_resupply_adherence = modify_reorder_historical(df_resupply_adherence)
df_resupply_adherence

,SetupDate,ResupplyDate,Year,MaskBrand,Compliance,PatientCount,DeviceCount,MaskCount,CushionCount,HeadgearCount,TubingCount,WaterChamberCount,ChinstrapCount,HumidifierCount,FilterCount,Year1Adherence,Year2Adherence,OngoingCompliance
0,2015-01-01,2015-01-01,1,airfit f10,0.817,60,63,119,176,100,121,27,1,59,421,0.607595,0.417722,0.687500
1,2015-01-01,2015-01-01,1,airfit n10,0.737,19,19,37,109,32,43,7,1,18,174,0.777778,0.555556,0.714286
2,2015-01-01,2015-01-01,1,airfit p10,0.891,55,57,109,333,98,103,23,27,55,446,0.725000,0.562500,0.775862
3,2015-01-01,2015-01-01,1,eson,1.000,8,9,15,20,14,15,5,4,8,37,0.875000,0.750000,0.857143
4,2015-01-01,2015-01-01,1,mirage fx,0.750,12,12,24,90,20,25,5,4,12,117,0.636364,0.636364,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716,2021-12-01,2021-12-01,1,mirage quattro,9.000,11,11,11,14,11,11,2,2,11,50,0.000000,0.000000,NaN
1717,2021-12-01,2021-12-01,1,quattro fx,9.000,3,3,4,2,2,3,0,0,3,10,0.000000,0.000000,NaN
1718,2021-12-01,2021-12-01,1,simplus,9.000,34,34,34,37,31,33,4,0,35,137,0.000000,0.000000,NaN
1719,2021-12-01,2021-12-01,1,swift fx,9.000,10,10,10,33,11,9,3,1,10,48,0.000000,0.000000,NaN


In [ ]:
# move historical resupply information data from latest to backup for other parts of project
df_resupply_folder_src = 'ltv_ljj_2021/analysis_output/historical_resupply/latest'
df_resupply_folder_target = 'ltv_ljj_2021/analysis_output/historical_resupply/backup'
move_delete_folder(bucket, df_resupply_folder_src, df_resupply_folder_target, False)

# write historical resupply information to s3
file_name = 'df_resupply_historical_setup_month_mask_' + str(dt_last_complete_month.date()) + '.csv'
df_resupply_mask_brand_dir = 's3://{}/{}/{}'.format(bucket, 'ltv_ljj_2021/analysis_output/historical_resupply/latest', file_name)
df_resupply_adherence.to_csv(df_resupply_mask_brand_dir, index=False)

ltv_ljj_2021/analysis_output/historical_resupply/latest/df_resupply_historical_setup_month_mask_2021-12-01.csv


In [ ]:
logging.info("Historical resupply and adherence by mask brand and setup month is done")

# 3 Perform naive forecest for setup month upto latest month

In [ ]:
##------------------------------------------------------------------
## 3 Perform naive forecest for setup month upto latest month
##------------------------------------------------------------------
logging.info("Naive forecast by mask brand and setup month starts")
dt_recent_first_year_resupply = global_last_so_dt + relativedelta(months=-13)
dt_recent_2nd_year_resupply = global_last_so_dt + relativedelta(months=-25)

## global last airview record date
global_last_av_dt = pd.to_datetime(min(df_selected['global_av_last_record_date'])).replace(day=1)
# setup month one year ago and two year ago for adherence
# those two dates are 1 month earlier than corresponding resupply dates because of dropout definition
# also last sale order date and last airview record date might be different
dt_recent_first_year_adherence = global_last_av_dt + relativedelta(months=-14)
dt_recent_2nd_year_adherence = global_last_av_dt + relativedelta(months=-26)

In [ ]:
global_last_av_dt

Timestamp('2021-12-01 00:00:00')

In [ ]:
dt_recent_first_year_adherence

Timestamp('2020-10-01 00:00:00')

In [ ]:
dt_recent_2nd_year_adherence

Timestamp('2019-10-01 00:00:00')

In [ ]:
# calculate weighted resupply rate and adherence rate(setup month one and two year ago) by mask category
df_weighted_average = get_weighted_average(df_resupply_adherence, df_selected, mask_infor)
df_weighted_average

,SetupDate,Year,MaskCategory,Compliance,DeviceCount,MaskCount,CushionCount,HeadgearCount,TubingCount,WaterChamberCount,ChinstrapCount,HumidifierCount,FilterCount,Year1Adherence,Year2Adherence,OngoingCompliance
0,2020-12-01,1,FF Mask,0.749587,1.01897,2.14171,3.98074,1.52199,2.12417,0.67232,0.0347801,1.00086,9.94079,0.587202,0,0
0,2019-12-01,1,FF Mask,0.756724,1.03033,2.19874,3.85356,1.5897,2.18881,0.678609,0.0251046,1.00497,10.2926,0.627434,0.485942,0.774491
0,2019-12-01,2,FF Mask,0.759504,0.021955,2.03084,4.57606,1.34501,1.96236,0.872974,0.0167277,0.00888657,9.89545,0.62817,0.486426,0.774354
0,2020-12-01,1,Nasal,0.782968,1.01618,2.12252,8.3111,1.52246,2.12483,0.616579,0.259577,0.997028,10.1182,0.650396,0,0
0,2019-12-01,1,Nasal,0.810339,1.02733,2.13598,8.22707,1.57124,2.14802,0.646714,0.23162,1.00423,10.2999,0.666456,0.53526,0.803145
0,2019-12-01,2,Nasal,0.811314,0.0174306,1.99871,9.66172,1.3215,1.96772,0.91672,0.136217,0.00387347,10.0329,0.666297,0.535056,0.80303
0,2020-12-01,1,Pillow,0.803016,1.01469,2.10847,8.36893,1.56271,2.09605,0.623164,0.293785,0.99322,9.95989,0.625642,0,0
0,2019-12-01,1,Pillow,0.808151,1.02606,2.18527,8.52295,1.62833,2.20227,0.669688,0.258924,1.01133,10.4448,0.666804,0.539168,0.808585
0,2019-12-01,2,Pillow,0.809335,0.0137143,2.112,9.93714,1.42743,2.05943,0.955429,0.158857,0.00457143,10.4331,0.667341,0.539707,0.808741


In [ ]:
# get first setup month with patient count > 100 for every mask brand
df_first_setup_month = get_first_month_with_large_setup_count(df_resupply_adherence, mask_infor)
df_first_setup_month

,SetupDate,MaskBrand
0,2015-06-01,airfit f10
1,2016-12-01,airfit f20
2,2018-11-01,airfit f30
3,2020-02-01,airfit f30i
4,2017-02-01,airfit n20
5,2019-12-01,airfit n30
6,2019-02-01,airfit n30i
7,2015-04-01,airfit p10
8,2019-05-01,airfit p30i
9,2017-07-01,airtouch f20


In [ ]:
df_resupply_adherence.head()

,SetupDate,ResupplyDate,Year,MaskBrand,Compliance,PatientCount,DeviceCount,MaskCount,CushionCount,HeadgearCount,TubingCount,WaterChamberCount,ChinstrapCount,HumidifierCount,FilterCount,Year1Adherence,Year2Adherence,OngoingCompliance
0,2015-01-01,2015-01-01,1,airfit f10,0.817,60,63,119,176,100,121,27,1,59,421,0.607595,0.417722,0.687500
1,2015-01-01,2015-01-01,1,airfit n10,0.737,19,19,37,109,32,43,7,1,18,174,0.777778,0.555556,0.714286
2,2015-01-01,2015-01-01,1,airfit p10,0.891,55,57,109,333,98,103,23,27,55,446,0.725000,0.562500,0.775862
3,2015-01-01,2015-01-01,1,eson,1.000,8,9,15,20,14,15,5,4,8,37,0.875000,0.750000,0.857143
4,2015-01-01,2015-01-01,1,mirage fx,0.750,12,12,24,90,20,25,5,4,12,117,0.636364,0.636364,1.000000


In [ ]:
# naive forecast for every mask brand
df_resupply_adherence_forecast = get_naive_forecast(df_resupply_adherence, df_selected, df_weighted_average, mask_infor, df_first_setup_month)
df_resupply_adherence_forecast

airfit f10
airfit f20


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


airfit f30


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


airfit n20


...

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


eson 2


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/indexing.py:1715: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


dreamwear gel pillow
dreamwisp
airtouch n20


,SetupDate,Year,ResupplyDate,MaskBrand,Compliance,PatientCount,DeviceCount,MaskCount,CushionCount,HeadgearCount,TubingCount,WaterChamberCount,ChinstrapCount,HumidifierCount,FilterCount,Year1Adherence,Year2Adherence,OngoingCompliance
0,2015-01-01,1,2015-01-01,airfit f10,0.817,60,63,119,176,100,121,27,1,59,421,0.607595,0.417722,0.687500
1,2015-01-01,2,2016-01-01,airfit f10,0.817,27,0,35,56,25,34,14,0,0,139,0.607595,0.417722,0.687500
2,2015-01-01,3,2017-01-01,airfit f10,0.817,24,1,42,52,33,36,19,0,0,124,0.607595,0.417722,0.687500
3,2015-01-01,4,2018-01-01,airfit f10,0.817,19,1,30,41,23,29,12,0,1,97,0.607595,0.417722,0.687500
4,2015-01-01,5,2019-01-01,airfit f10,0.817,18,1,27,45,23,23,8,0,1,94,0.607595,0.417722,0.687500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,2021-12-01,2,2022-12-01,airtouch n20,9.000,24,0,49,238,32,48,22,3,0,247,0.650396,0.522362,0.803145
28,2020-09-01,2,2021-09-01,airtouch n20,1.000,3,0,6,29,4,6,3,0,0,30,0.750000,0.602358,0.803145
29,2020-10-01,2,2021-10-01,airtouch n20,0.839,21,0,42,201,28,41,19,3,0,209,0.672131,0.539819,0.803145
30,2020-11-01,2,2021-11-01,airtouch n20,0.783,30,1,60,289,40,59,27,4,0,300,0.650396,0.522362,0.803145


In [ ]:
# modify column names, add columns and reorder columns
df_resupply_adherence_forecast = modify_reorder_forecast(df_resupply_adherence_forecast, mask_infor)

In [ ]:
df_resupply_adherence_forecast.head()

,MaskBrand,SetupDate,Year,ResupplyDate,Compliance,PatientCount,DeviceCount,MaskCount,CushionCount,HeadgearCount,TubingCount,WaterChamberCount,ChinstrapCount,HumidifierCount,FilterCount,Year1Adherence,Year2Adherence,OngoingCompliance,MaskCategory,MaskManuf
0,AirFit F10,2015-01-01,1,2015-01-01,0.817,60,63,119,176,100,121,27,1,59,421,0.607595,0.417722,0.6875,FF Mask,ResMed
1,AirFit F10,2015-01-01,2,2016-01-01,0.817,27,0,35,56,25,34,14,0,0,139,0.607595,0.417722,0.6875,FF Mask,ResMed
2,AirFit F10,2015-01-01,3,2017-01-01,0.817,24,1,42,52,33,36,19,0,0,124,0.607595,0.417722,0.6875,FF Mask,ResMed
3,AirFit F10,2015-01-01,4,2018-01-01,0.817,19,1,30,41,23,29,12,0,1,97,0.607595,0.417722,0.6875,FF Mask,ResMed
4,AirFit F10,2015-01-01,5,2019-01-01,0.817,18,1,27,45,23,23,8,0,1,94,0.607595,0.417722,0.6875,FF Mask,ResMed


In [ ]:
# move naive_forecast from latest to backup
naive_forecast_folder_src = 'ltv_ljj_2021/analysis_output/resupply_with_naive_forecast/latest/'
naive_forecast_folder_target = 'ltv_ljj_2021/analysis_output/resupply_with_naive_forecast/backup/'
move_delete_folder(bucket, naive_forecast_folder_src, naive_forecast_folder_target, False)

# write historical and naive forecast to s3
year = dt_last_complete_month.date().year
month = dt_last_complete_month.date().month
file_name = 'df_resupply_adherence_naive_forecast_mask_brand_' + str(year) + '_' + str(month) + '.csv'
naive_forecast_dir = 's3://{}/{}/{}'.format(bucket, naive_forecast_folder_src, file_name)
df_resupply_adherence_forecast.to_csv(naive_forecast_dir, index=False)

ltv_ljj_2021/analysis_output/resupply_with_naive_forecast/latest/df_resupply_adherence_naive_forecast_mask_brand_2021-12-01.csv


In [ ]:
# delete naive forecast from naive forecast folder to generate table in athena
table_naive_forecast_folder = 'ltv_ljj_2021/analysis_output/performance_metrics_and_forecast/naive_forecast'
delete_folder(bucket, table_naive_forecast_folder)

# write naive forecast to naive forecast folder in s3
file_name = 'naive_forecast_' + str(year) + '_' + str(month) + '.csv'
naive_forecast_dir = 's3://{}/{}/{}'.format(bucket, table_naive_forecast_folder, file_name)
df_resupply_adherence_forecast.to_csv(naive_forecast_dir, index=False)

In [ ]:
# wait for the file to be saved in s3
logging.info("Naive forecast by mask brand and setup month is done")

In [ ]:
# save a copy this notebook to s3 after the run
file_name = 'ltv_naive_forecast_' + str(year) + '_' + str(month) + '.ipynb'
!aws s3 cp /home/ec2-user/SageMaker/ltv-lijj-2021/ltv_naive_forecast/naive_forecast_test.ipynb  s3://$bucket/ltv_ljj_2021/scheduled_notebooks/$file_name

In [ ]:
##-------------------------------------------------------------------------
## 4 delete previous table for df_resupply_adherence_forecast in athena and
## call glue crawler to generate new table
##-------------------------------------------------------------------------
# logging.info("Glue cawler job starts")
# # delete previous table
# athena_client.start_query_execution(
#     QueryString='DROP TABLE IF EXISTS resupply_adherence.latest;',
#     ResultConfiguration={
#         'OutputLocation': "s3://{}/{}".format(bucket, athena_query_target),
#     }
# )
# # logging.info("Glue cawler job starts")
# glue_client = boto3.client('glue')
# # run glue crawler
# glue_client.start_crawler(Name='ltv-naive-forecast-resupply-adherence')

{'ResponseMetadata': {'RequestId': '1d6d66fd-9ce8-42a0-8d1f-39707293ba0a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 19 Jan 2022 08:20:52 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1d6d66fd-9ce8-42a0-8d1f-39707293ba0a'},
  'RetryAttempts': 0}}

In [ ]:
logging.info("Glue cawler job is done")

CrawlerNotRunningException: An error occurred (CrawlerNotRunningException) when calling the StopCrawler operation: Crawler with name ltv-naive-forecast-resupply-adherence is not running

In [ ]:
# relook the code in 2025